## Instalations

In [ ]:
!pip install tqdm
!pip install click
!pip install -q -U keras-tuner
import tensorflow as tf
import keras
import keras_tuner
!pip install keras-tuner[bayesian]
!pip install scipy

## Global variables

In [ ]:
G_cipher = None
G_data_format = None
G_network = None
G_shape = None
G_tune = None
G_num_epochs = None

In [ ]:
def set_global_variables(cipher, data_format, network, shape, tune, num_epochs):
  global G_cipher
  global G_data_format
  global G_network
  global G_shape
  global G_tune
  global G_num_epochs

  G_cipher = cipher
  G_data_format = data_format
  G_network = network
  G_shape = shape
  G_tune = tune
  G_num_epochs = num_epochs

In [ ]:
def reset_global_variables():
  global G_cipher
  global G_data_format
  global G_network
  global G_shape
  global G_tune
  global G_num_epochs

  G_cipher = None
  G_data_format = None
  G_network = None
  G_shape = None
  G_tune = None
  G_num_epochs = None

In [ ]:
def print_global_variables():
  global G_cipher
  global G_data_format
  global G_network
  global G_shape
  global G_tune
  global G_num_epochs

  if G_cipher is not None:
    print("Cipher: " + str(G_cipher.get_name()))
  else:
    print(None)

  if G_data_format is not None:
    print("Data format: " + G_data_format)
  else:
    print(None)

  if G_network is not None:
    print("Network: " + G_network)
  else:
    print(None)

  if G_shape is not None:
    print("Input shape: " + G_shape)
  else:
    print(None)

  if G_tune is not None:
    print("Tuning: " + str(G_tune))
  else:
    print(None)

  if G_num_epochs is not None:
    print("Training epochs: " + str(G_num_epochs))
  else:
    print(None)

## Ciphers

Based on code from: https://github.com/differential-neural/An-Assessment-of-Differential-Neural-Distinguishers

In [ ]:
class NotImplementedException(Exception):
    def __init__(self, func, cls):
        self.func = func
        self.cls = cls
        super().__init__(f"Function {func} (of class {cls}) is not implemented!")

In [ ]:
from abc import ABC, abstractmethod
import numpy as np
from os import urandom



class AbstractCipher(ABC):
    """ Abstract cipher class containing all methods a cipher class should implement """

    """ Data types for all the supported word sizes """
    DTYPES = {
        2: np.uint8,
        4: np.uint8,
        8: np.uint8,
        16: np.uint16,
        32: np.uint32
    }

    def __init__(
            self, n_rounds, word_size, n_words, n_main_key_words, n_round_key_words,
            use_key_schedule=True, main_key_word_size=None, round_key_word_size=None
    ):
        """
        Initializes a cipher object
        :param n_rounds: The number of rounds used for de-/encryption
        :param word_size: The size (in bits) of a ciphertext word
        :param n_words: The number of words of one ciphertext
        :param n_main_key_words: The number of words in the main key
        :param n_round_key_words: The number of words in each round key
        :param use_key_schedule: Whether to use the key schedule or independent round keys
        :param main_key_word_size: The size (in bits) of a main key word ('None' means the same as word_size)
        :param round_key_word_size: The size (in bits) of a round key word ('None' means the same as word_size)
        """
        self.n_rounds = n_rounds
        self.word_size = word_size
        self.word_dtype = self.DTYPES.get(self.word_size, None)
        if self.word_dtype is None:
            raise Exception(f'Error: Unexpected word size {self.word_size}')
        self.mask_val = 2 ** self.word_size - 1
        self.n_words = n_words
        self.n_main_key_words = n_main_key_words
        self.n_round_key_words = n_round_key_words
        self.use_key_schedule = use_key_schedule
        self.main_key_word_size = main_key_word_size if main_key_word_size is not None else word_size
        self.main_key_word_dtype = self.DTYPES.get(self.main_key_word_size, None)
        if self.main_key_word_dtype is None:
            raise Exception(f'Error: Unexpected word size {self.main_key_word_size}')
        self.round_key_word_size = round_key_word_size if round_key_word_size is not None else word_size
        self.round_key_word_dtype = self.DTYPES.get(self.round_key_word_size, None)
        if self.round_key_word_dtype is None:
            raise Exception(f'Error: Unexpected word size {self.round_key_word_size}')


    def get_word_size(self):
        """
        :return: The size (in bits) of one word (which could be the size of an s-box or of the right/left side)
        """
        return self.word_size

    def get_n_words(self):
        """
        :return: The number of words in one ciphertext
        """
        return self.n_words

    def get_block_size(self):
        """
        :return: The size (in bits) of one ciphertext
        """
        return self.word_size * self.n_words

    def get_n_rounds(self):
        """
        :return: The number of rounds
        """
        return self.n_rounds

    def set_n_rounds(self, new_n_rounds):
        """
        Sets the number of rounds
        :param new_n_rounds: The new number of rounds
        """
        self.n_rounds = new_n_rounds

    @staticmethod
    def bytes_per_word(word_size):
        """
        :param word_size: The word size (in bits)
        :return: Returns the number of bytes to represent a word of word_size bits
        """
        return word_size // 8 + (1 if (word_size % 8) else 0)

    @abstractmethod
    def encrypt_one_round(self, p, k, rc=None):
        """
        Round function of the cipher
        :param p: The plaintext
        :param k: The round key
        :param rc: The round constant
        :return: The one round encryption of p using key k
        """
        pass

    def encrypt(self, p, keys):
        """
        Encrypt by applying the round function for each given round key
        :param p: The plaintext
        :param keys: A list of round keys
        :return: The encryption of p under the round keys in keys
        """
        state = p
        for i in range(len(keys)):
            state = self.encrypt_one_round(state, keys[i], self.get_rc(i))
        return state

    @abstractmethod
    def decrypt_one_round(self, c, k, rc=None):
        """
        Inverse round function of the cipher
        :param c: The ciphertext
        :param k: The round key
        :param rc: The round constant
        :return: The one round decryption of c using key k
        """
        pass

    def decrypt(self, c, keys):
        """
        Decrypt by applying the inverse round function for each given key
        :param c: The ciphertext
        :param keys: A list of round keys
        :return: The decryption of c under the round keys in keys
        """
        state = c
        for i in range(len(keys) - 1, -1, -1):
            state = self.decrypt_one_round(state, keys[i], self.get_rc(i))
        return state

    @abstractmethod
    def calc_back(self, c, p=None, variant=1):
        """
        Revert deterministic parts of the round function
        :param c: The ciphertext
        :param p: The initial plaintext
        :param variant: Select the variant of how to calculate back (default is 1; 0 means not calculating back)
        :return: The inner state after reverting the deterministic transformation at the end of the encryption process
        """
        pass

    def get_rc(self, r):
        """
        :param r: The round
        :return: The round constant for round r
        """
        return None

    def draw_keys(self, n_samples):
        """
        :param n_samples: How many keys to draw
        :return: An array of keys
        """
        if self.use_key_schedule:
            bytes_per_word = self.bytes_per_word(self.main_key_word_size)
            main_key = np.frombuffer(
                urandom(self.n_main_key_words * bytes_per_word * n_samples), dtype=self.main_key_word_dtype
            ).reshape(self.n_main_key_words, n_samples)
            if self.main_key_word_size < 8:
                # Note: If the word size is greater than 8, it will always fit the dtype for the ciphers we use
                main_key = np.right_shift(main_key, 8 - self.main_key_word_size)
            return self.key_schedule(main_key)
        else:
            bytes_per_word = self.bytes_per_word(self.round_key_word_size)
            round_keys = np.frombuffer(
                urandom(self.n_rounds * self.n_round_key_words * bytes_per_word * n_samples),
                dtype=self.round_key_word_dtype
            ).reshape(self.n_rounds, self.n_round_key_words, n_samples)
            if self.round_key_word_size < 8:
                # Note: If the word size is greater than 8, it will always fit the dtype for the ciphers we use
                round_keys = np.right_shift(round_keys, 8 - self.round_key_word_size)
            return round_keys

    def draw_plaintexts(self, n_samples):
        """
        :param n_samples: How many plaintexts to draw
        :return: An array of plaintexts
        """
        # In most cases the format of the plain- and ciphertexts are the same,
        # so we can return random ciphertexts at this point
        return self.draw_ciphertexts(n_samples)

    def draw_ciphertexts(self, n_samples):
        """
        :param n_samples: How many ciphertexts to draw
        :return: An array of ciphertexts
        """
        bytes_per_word = self.bytes_per_word(self.word_size)
        ct = np.reshape(
            np.frombuffer(urandom(bytes_per_word * self.n_words * n_samples), dtype=self.word_dtype),
            (self.n_words, n_samples)
        )
        if self.word_size < 8:
            # Note: If the word size is greater than 8, it will always fit the dtype for the ciphers we use
            ct = np.right_shift(ct, 8 - self.word_size)
        return ct

    @abstractmethod
    def key_schedule(self, key):
        """
        Applies the key schedule
        :param key: The key
        :return: A list of round keys
        """
        pass

    def rol(self, x, k):
        """
        :param x: What to rotate
        :param k: How to rotate
        :return: x rotated by k bits to the left
        """
        return ((x << k) & self.mask_val) | (x >> (self.word_size - k))

    def ror(self, x, k):
        """
        :param x: What to rotate
        :param k: How to rotate
        :return: x rotated by k bits to the right
        """
        return (x >> k) | ((x << (self.word_size - k)) & self.mask_val)

    @staticmethod
    @abstractmethod
    def get_test_vectors():
        """
        :return: Returns the test vectors used for verifying the correct implementation of the cipher as a list of
            tuples of the form (cipher, plaintext, key, ciphertext), where cipher is an instance of self that will
            be used to verify the test vector
        """
        pass

    @classmethod
    def verify_test_vectors(cls):
        """
        Verifies the test vectors given by the designers
        :return: Result of the test
        """
        for cipher, pt, key, ct in cls.get_test_vectors():
            if not np.array_equal(ct, cipher.encrypt(pt, key)):
                print(f"ERROR: Test vector for {cls.__name__} not verified (encryption did not match)")
                return False
            try:  # This will use decrypt, which may not be implemented
                if not np.array_equal(pt, cipher.decrypt(ct, key)):
                    print(f"ERROR: Test vector for {cls.__name__} not verified (decryption did not match)")
                    return False
            except NotImplementedException as e:
                print(f"Info: Decryption not implemented for {cls.__name__}. (Original message: {e})")

        print(f"Info: All test vectors for {cls.__name__} verified")
        return True

In [ ]:
import numpy as np


class Speck(AbstractCipher):

    def __init__(self, n_rounds=22, word_size=16, use_key_schedule=True, alpha=7, beta=2, m=4):
        """
        Initializes a Speck cipher object
        :param n_rounds: The number of rounds
        :param word_size: The size (in bits) of the right/left side
        :param use_key_schedule: Whether to use the key schedule or independent round keys
        :param alpha: The rotational constant alpha
        :param beta: The rotational constant beta
        :param m: Number of words of the key
        """
        super(Speck, self).__init__(
            n_rounds, word_size, n_words=2, n_main_key_words=m, n_round_key_words=1, use_key_schedule=use_key_schedule
        )
        self.alpha = alpha
        self.beta = beta

    @staticmethod
    def get_name():
      return "Speck"

    def encrypt_one_round(self, p, k, rc=None):
        """
        Round function of the cipher
        :param p: The plaintext
        :param k: The round key
        :param rc: The round constant
        :return: The one round encryption of p using key k
        """
        c0, c1 = p[0], p[1]
        c0 = self.ror(c0, self.alpha)
        c0 = (c0 + c1) & self.mask_val
        c0 = c0 ^ k[0]  # round key consists of one word only
        c1 = self.rol(c1, self.beta)
        c1 = c1 ^ c0
        return c0, c1

    def decrypt_one_round(self, c, k, rc=None):
        """
        Inverse round function of the cipher
        :param c: The ciphertext
        :param k: The round key
        :param rc: The round constant
        :return: The one round decryption of c using key k
        """
        c0, c1 = c[0], c[1]
        c1 = c1 ^ c0
        c1 = self.ror(c1, self.beta)
        c0 = c0 ^ k[0]  # round key consists of one word only
        c0 = (c0 - c1) & self.mask_val
        c0 = self.rol(c0, self.alpha)
        return c0, c1

    def calc_back(self, c, p=None, variant=1):
        """
        Revert deterministic parts of the round function
        :param c: The ciphertext
        :param p: The initial plaintext
        :param variant: Select the variant of how to calculate back (default is 1; 0 means not calculating back)
        :return: The inner state after reverting the deterministic transformation at the end of the encryption process
        """
        if variant == 0:
            return c
        if variant != 1:
            raise Exception(f'ERROR: Variant {variant} of calculating back is not implemented')
        c0, c1 = c[0], c[1]
        c1 = c1 ^ c0
        c1 = self.ror(c1, self.beta)
        return c0, c1

    def key_schedule(self, key):
        """
        Applies the key schedule
        :param key: The key
        :return: A list of round keys
        """
        ks = [0 for i in range(self.n_rounds)]
        ks[0] = key[len(key)-1]
        l = list(reversed(key[:len(key)-1]))

        for i in range(self.n_rounds-1):
            l[i % 3], ks[i+1] = self.encrypt_one_round((l[i % 3], ks[i]), [i])

        return np.array(ks, dtype=self.main_key_word_dtype)[:, np.newaxis]

    @staticmethod
    def get_test_vectors():
        """
        Test vectors from https://eprint.iacr.org/2013/404.pdf, page 42
        :return: Returns the test vectors used for verifying the correct implementation of the cipher as a list of
            tuples of the form (cipher, plaintext, key, ciphertext), where cipher is an instance of self that will
            be used to verify the test vector
        """
        # Initialize Speck32/64 according to specification
        speck32_64 = Speck()
        key = np.array([[0x1918], [0x1110], [0x0908], [0x0100]], dtype=np.uint16)
        ks = speck32_64.key_schedule(key)
        pt = np.array([[0x6574], [0x694c]], dtype=np.uint16)
        ct = np.array([[0xa868], [0x42f2]], dtype=np.uint16)
        return [(speck32_64, pt, ks, ct)]

In [ ]:
import numpy as np


class Simon(AbstractCipher):

    def __init__(self, n_rounds=32, word_size=16, m=4, alpha=1, beta=8, gamma=2, const_seq=0, use_key_schedule=True):
        """
        Initializes a Simon cipher object
        :param n_rounds: The number of rounds used for de-/encryption
        :param word_size:
        :param m:
        :param alpha:
        :param beta:
        :param gamma:
        :param const_seq:
        :param use_key_schedule:
        """
        super(Simon, self).__init__(
            n_rounds, word_size, n_words=2, n_main_key_words=m, n_round_key_words=1, use_key_schedule=use_key_schedule
        )
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.const_seq = const_seq

    @staticmethod
    def get_name():
      return "Simon"

    def encrypt_one_round(self, p, k, rc=None):
        """
        Round function of the cipher
        :param p: The plaintext
        :param k: The round key
        :param rc: The round constant
        :return: The one round encryption of p using key k
        """
        c0 = (((self.rol(p[0], self.alpha) & self.rol(p[0], self.beta)) ^ self.rol(p[0], self.gamma)) ^ p[1]) ^ k[0]
        c1 = p[0]
        return c0, c1

    def decrypt_one_round(self, c, k, rc=None):
        """
        Inverse round function of the cipher
        :param c: The ciphertext
        :param k: The round key
        :param rc: The round constant
        :return: The one round decryption of c using key k
        """
        c0, c1 = c[0], c[1]
        c0 = ((c0 ^ k[0]) ^ self.rol(c1, self.gamma)) ^ (self.rol(c1, self.alpha) & self.rol(c1, self.beta))
        return c1, c0

    def calc_back(self, c, p=None, variant=1):
        """
        Revert deterministic parts of the round function
        :param c: The ciphertext
        :param p: The initial plaintext
        :param variant: Select the variant of how to calculate back (default is 1; 0 means not calculating back)
        :return: The inner state after reverting the deterministic transformation at the end of the encryption process
        """
        if variant == 0:
            return c
        if variant != 1:
            raise Exception("ERROR: Only one variant of calculating back is known")
        c0, c1 = c[0], c[1]
        c0 = (c0 ^ self.rol(c1, self.gamma)) ^ (self.rol(c1, self.alpha) & self.rol(c1, self.beta))
        return c1, c0

    def key_schedule(self, key):
        """
        Applies the key schedule
        :param key: The key
        :return: A list of round keys
        """
        z = [[1,1,1,1,1,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,1,0,1,1,1,1,1,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,1,0],
             [1,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,0,0,1,1,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,0,0,1,1,0,0,0,0,1,0,1,1,0,1,0],
             [1,0,1,0,1,1,1,1,0,1,1,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,1,0,1,1,0,1,1,0,0,1,1],
             [1,1,0,1,1,0,1,1,1,0,1,0,1,1,0,0,0,1,1,0,0,1,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,1,1,0,0,1,1,0,1,0,0,0,0,1,1,1,1],
             [1,1,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,1,1,0,0,1,0,1,0,0,1,0,0,1,1,1,0,1,1,1,1]]
        ks = [0 for i in range(self.n_rounds)]
        ks[:self.n_main_key_words] = list(reversed(key))
        for i in range(self.n_main_key_words, self.n_rounds):
            tmp = self.ror(ks[i-1], 3)
            if self.n_main_key_words == 4:
                tmp ^= ks[i-3]
            tmp ^= self.ror(tmp, 1)
            ks[i] = (ks[i-self.n_main_key_words] ^ self.mask_val) ^ tmp \
                    ^ z[self.const_seq][(i-self.n_main_key_words) % 62] ^ 3
        return np.array(ks, dtype=self.main_key_word_dtype)[:, np.newaxis]

    @staticmethod
    def get_test_vectors():
        """
        Test vectors from https://eprint.iacr.org/2013/404.pdf, page 41
        :return: Returns the test vectors used for verifying the correct implementation of the cipher as a list of
            tuples of the form (cipher, plaintext, key, ciphertext), where cipher is an instance of self that will
            be used to verify the test vector
        """
        # Initialize Simon32/64 according to specification
        simon32_64 = Simon()
        key = np.array([[0x1918], [0x1110], [0x0908], [0x0100]], dtype=np.uint16)
        ks = simon32_64.key_schedule(key)
        pt = np.array([[0x6565], [0x6877]], dtype=np.uint16)
        ct = np.array([[0xc69b], [0xe9bb]], dtype=np.uint16)

        # Initialize Simon64/128 according to specification
        simon64_128 = Simon(n_rounds=44, word_size=32, alpha=1, beta=8, gamma=2, const_seq=3)
        key = np.array([[0x1b1a1918], [0x13121110], [0x0b0a0908], [0x03020100]], dtype=np.uint32)
        ks2 = simon64_128.key_schedule(key)
        pt2 = np.array([[0x656b696c], [0x20646e75]], dtype=np.uint32)
        ct2 = np.array([[0x44c8fc20], [0xb9dfa07a]], dtype=np.uint32)

        return [(simon32_64, pt, ks, ct), (simon64_128, pt2, ks2, ct2)]

In [ ]:
import numpy as np


class Skinny(AbstractCipher):

    def __init__(self, n_rounds=32, use_key_schedule=True):
        """
        Initializes a Skinny cipher object
        :param n_rounds: The number of round used for de-/encryption
        :param use_key_schedule: Whether to use the key schedule or independent round keys
        """
        super(Skinny, self).__init__(
            n_rounds, word_size=4, n_words=16, n_main_key_words=16, n_round_key_words=16,
            use_key_schedule=use_key_schedule
        )

    # Index:             0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f
    SBOX    = np.array([0xC, 0x6, 0x9, 0x0, 0x1, 0xa, 0x2, 0xb, 0x3, 0x8, 0x5, 0xd, 0x4, 0xe, 0x7, 0xf])
    SBOXINV = np.array([0x3, 0x4, 0x6, 0x8, 0xC, 0xa, 0x1, 0xe, 0x9, 0x2, 0x5, 0x7, 0x0, 0xb, 0xd, 0xf])

    @staticmethod
    def get_name():
        return "Skinny"

    @staticmethod
    def substitute(state, sb):
        """
        Applies the s-box sb on each word/nibble of state
        :param state: The state to act on
        :param sb: The s-box
        :return: The resulting state
        """
        result = []
        for s in state:
            result.append(sb[s])
        return result

    def substitution_layer(self, state):
        """
        Applies the substitution layer to state
        :param state: The current state
        :return: The resulting state
        """
        return self.substitute(state, self.SBOX)

    def inv_substitution_layer(self, state):
        """
        Applies the inverse substitution layer to state
        :param state: The current state
        :return: The resulting state
        """
        return self.substitute(state, self.SBOXINV)

    @staticmethod
    def add_constants(state, constants):
        """
        Adds the round constant to state. Constants should contain the values of rc0 to rc5, where rc0 is the lsb
        :param state: The current state
        :param constants: The round constant
        :return: The resulting state
        """
        # Add rc0 to rc3 to the first word/nibble of the state
        state[0] = state[0] ^ (constants & 0xf)
        # Add rc4 and rc5 to the first word/nibble of the second row of the state
        state[4] = state[4] ^ (constants >> 4)
        # Add c2=0x2 to the first word/nibble of the third row of the state
        state[8] = state[8] ^ 0x2
        return state

    @staticmethod
    def add_tweak_key(state, tweakKey):
        """
        Adds the (tweak-)key to the first two rows of the state
        :param state: The current state
        :param tweakKey: The tweak-key to add
        :return: the resulting state
        """
        for i in range(8):
            state[i] = state[i] ^ tweakKey[i]
        return state

    @staticmethod
    def shift_rows(state):
        """
        :param state: The current state
        :return: The state with the rows shifted
        """
        return [state[0], state[1], state[2], state[3],
                state[7], state[4], state[5], state[6],
                state[10], state[11], state[8], state[9],
                state[13], state[14], state[15], state[12]]

    @staticmethod
    def inv_shift_rows(state):
        """
        :param state: The current state
        :return: The state with the rows shifted
        """
        return [state[0], state[1], state[2], state[3],
                state[5], state[6], state[7], state[4],
                state[10], state[11], state[8], state[9],
                state[15], state[12], state[13], state[14]]

    @staticmethod
    def mix_columns(state):
        """
        Applies the mix columns operation
        :param state: The current state
        :return: The resulting state
        """
        # Add 3rd row to 2nd row
        for i in range(4):
            state[4 + i] = state[4 + i] ^ state[8 + i]
        # Add 1st row to 3rd row
        for i in range(4):
            state[8 + i] = state[8 + i] ^ state[i]
        # Add 3rd row to 4th row
        for i in range(4):
            state[12 + i] = state[12 + i] ^ state[8 + i]
        # Permute columns and return
        return [state[12], state[13], state[14], state[15],
                state[0], state[1], state[2], state[3],
                state[4], state[5], state[6], state[7],
                state[8], state[9], state[10], state[11]]

    @staticmethod
    def inv_mix_columns(state):
        """
        Applies the inverse mix columns operation
        :param state: The current state
        :return: The resulting state
        """
        # Revert permutation of columns
        state = [state[4], state[5], state[6], state[7],
                 state[8], state[9], state[10], state[11],
                 state[12], state[13], state[14], state[15],
                 state[0], state[1], state[2], state[3]]
        # Add 3rd row to 4th row
        for i in range(4):
            state[12 + i] = state[12 + i] ^ state[8 + i]
        # Add 1st row to 3rd row
        for i in range(4):
            state[8 + i] = state[8 + i] ^ state[i]
        # Add 3rd row to 2nd row
        for i in range(4):
            state[4 + i] = state[4 + i] ^ state[8 + i]
        return state

    def encrypt_one_round(self, p, k, rc=None):
        """
        Round function of the cipher
        :param p: The plaintext
        :param k: The round key
        :param rc: The round constant
        :return: The one round encryption of p using key k
        """
        if rc is None:
            raise Exception("ERROR: Round constant has to be set for Skinny encryption")
        s = self.substitution_layer(p)
        s = self.add_constants(s, rc)
        s = self.add_tweak_key(s, k)
        s = self.shift_rows(s)
        s = self.mix_columns(s)
        return s

    def decrypt_one_round(self, c, k, rc=None):
        """
        Inverse round function of the cipher
        :param c: The ciphertext
        :param k: The round key
        :param rc: The round constant
        :return: The one round decryption of c using key k
        """
        if rc is None:
            raise Exception("ERROR: Round constant has to be set for Skinny decryption")
        s = self.inv_mix_columns(c)
        s = self.inv_shift_rows(s)
        s = self.add_tweak_key(s, k)
        s = self.add_constants(s, rc)
        s = self.inv_substitution_layer(s)
        return s

    def calc_back(self, c, p=None, variant=1):
        """
        Revert deterministic parts of the round function
        :param c: The ciphertext
        :param p: The initial plaintext
        :param variant: Select the variant of how to calculate back (default is 1; 0 means not calculating back)
            Options are
                - 1: Only revert Mix Columns and Shift Rows
                - 2: Revert Mix Columns, Shift Rows, Add Constants and the SBoxes for last two rows
        :return: The inner state after reverting the deterministic transformation at the end of the encryption process
        """
        if variant == 0:
            return c
        # Always revert Mix Columns and Shift Rows
        s = self.inv_mix_columns(c)
        s = self.inv_shift_rows(s)
        if variant == 1:  # Only revert Mix Columns and Shift Rows
            return s
        if variant == 2:  # Revert Mix Columns, Shift Rows, Add Constants and the SBoxes for last two rows
            # Remove round constants from last round
            s = self.add_constants(s, self.get_rc(self.n_rounds - 1))
            # Revert SBoxes not influenced by the round (tweak-)key
            for i in range(8, 16):
                s[i] = self.SBOXINV[s[i]]
            return s
        raise Exception(f'ERROR: Variant {variant} of calculating back is not implemented')

    def get_rc(self, r):
        """
        :param r: The round
        :return: The round constant for round r
        """
        constant = 0x1
        for key in range(r):
            # Update constant
            constant = ((constant << 1) & 0x3f) ^ ((constant >> 5) & 1) ^ ((constant >> 4) & 1) ^ 1
        return constant

    def key_schedule(self, key):
        """
        Applies the key schedule
        :param key: The key
        :return: A list of round keys
        """
        ks = [key[:8]]
        for i in range(self.n_rounds - 1):
            # Permute tweak-key state by P_T
            key = [key[i] for i in [9,15,8,13,10,14,12,11,0,1,2,3,4,5,6,7]]
            # Add first two rows as round (tweak-)key
            ks.append(key[:8])
        # Bring round keys in the form (roundConstant, roundKey) and return
        return ks

    @staticmethod
    def get_test_vectors():
        """
        Test vectors from https://eprint.iacr.org/2016/660.pdf, page 46
        :return: Returns the test vectors used for verifying the correct implementation of the cipher as a list of
            tuples of the form (cipher, plaintext, key, ciphertext), where cipher is an instance of self that will
            be used to verify the test vector
        """
        # Initialize Skinny64/64 according to specification
        skinny_64_64 = Skinny()
        key = np.array([[0xf], [0x5], [0x2], [0x6], [0x9], [0x8], [0x2], [0x6],
                        [0xf], [0xc], [0x6], [0x8], [0x1], [0x2], [0x3], [0x8]], dtype=np.uint8)
        ks = skinny_64_64.key_schedule(key)
        pt = np.array([[0x0], [0x6], [0x0], [0x3], [0x4], [0xf], [0x9], [0x5],
                       [0x7], [0x7], [0x2], [0x4], [0xd], [0x1], [0x9], [0xd]], dtype=np.uint8)
        ct = np.array([
            [0xb], [0xb], [0x3], [0x9], [0xd], [0xf], [0xb], [0x2],
            [0x4], [0x2], [0x9], [0xb], [0x8], [0xa], [0xc], [0x7]
        ], dtype=np.uint8)
        return [(skinny_64_64, pt, ks, ct)]

In [ ]:
import numpy as np


class Present(AbstractCipher):

    def __init__(self, n_rounds=31, use_key_schedule=True):
        """
        Initializes a Present cipher object
        :param n_rounds: The number of round used for de-/encryption
        :param use_key_schedule: Whether to use the key schedule or independent round keys
        """
        super(Present, self).__init__(
            n_rounds + 1,  # Internally, we will use the number of rounds as the number of round key additions
            word_size=4, n_words=16, n_main_key_words=20, n_round_key_words=16, use_key_schedule=use_key_schedule
        )

    # Index:             0    1    2    3    4    5    6    7    8    9    A    B    C    D    E    F
    SBOX    = np.array([0xC, 0x5, 0x6, 0xB, 0x9, 0x0, 0xA, 0xD, 0x3, 0xE, 0xF, 0x8, 0x4, 0x7, 0x1, 0x2])
    SBOXINV = np.array([0x5, 0xE, 0xF, 0x8, 0xC, 0x1, 0x2, 0xD, 0xB, 0x4, 0x6, 0x3, 0x0, 0x7, 0x9, 0xA])

    @staticmethod
    def get_name():
        return "Present"

    def get_n_rounds(self, key_additions=True):
        """
        :param key_additions: Whether to return the number of round key additions or the number of rounds
        :return: The number of rounds or the number of round key additions
        """
        if key_additions:
            return super(Present, self).get_n_rounds()
        return super(Present, self).get_n_rounds() - 1

    @staticmethod
    def bit_at_pos(x, i):
        """
        :param x: Array
        :param i: Bit position to return
        :return: The i-th bit of all elements in x
        """
        return np.right_shift(x, i) & 0b1

    @staticmethod
    def bit_to_pos(x, i):
        """
        :param x: Array
        :param i: Bit position
        :return: All elements of x shifted to the left by i bits
        """
        return np.left_shift(x, i)

    def encrypt_one_round(self, p, k, rc=None):
        """
        Round function of the cipher
        :param p: The plaintext
        :param k: The round key
        :param rc: The round constant
        :return: The one round encryption of p using key k
        """
        s = [np.zeros(p[i].shape, dtype=self.word_dtype) for i in range(16)]
        for i in range(16):
            # add round key
            s[i] = p[i] ^ k[i]
            # apply s-box layer
            s[i] = self.SBOX[s[i]]
        c = [np.zeros(s[i].shape, dtype=self.word_dtype) for i in range(16)]
        # permutation layer
        for i in range(4):
            q = 3-i
            c[4*i+0] = self.bit_to_pos(self.bit_at_pos(s[0], q), 3) + self.bit_to_pos(self.bit_at_pos(s[1], q), 2) + \
                       self.bit_to_pos(self.bit_at_pos(s[2], q), 1) + self.bit_to_pos(self.bit_at_pos(s[3], q), 0)

            c[4*i+1] = self.bit_to_pos(self.bit_at_pos(s[4], q), 3) + self.bit_to_pos(self.bit_at_pos(s[5], q), 2) + \
                       self.bit_to_pos(self.bit_at_pos(s[6], q), 1) + self.bit_to_pos(self.bit_at_pos(s[7], q), 0)

            c[4*i+2] = self.bit_to_pos(self.bit_at_pos(s[8], q), 3) + self.bit_to_pos(self.bit_at_pos(s[9], q), 2) + \
                       self.bit_to_pos(self.bit_at_pos(s[10], q), 1) + self.bit_to_pos(self.bit_at_pos(s[11], q), 0)

            c[4*i+3] = self.bit_to_pos(self.bit_at_pos(s[12], q), 3) + self.bit_to_pos(self.bit_at_pos(s[13], q), 2) + \
                       self.bit_to_pos(self.bit_at_pos(s[14], q), 1) + self.bit_to_pos(self.bit_at_pos(s[15], q), 0)
        return c

    def encrypt(self, p, keys):
        """
        Encrypt by applying the round function for each given round key
        :param p: The plaintext
        :param keys: A list of round keys
        :return: The encryption of p under the round keys in keys
        """
        # For Present, the number of round keys is the number of rounds + 1
        c = super(Present, self).encrypt(p, keys[:-1])
        return c ^ keys[-1]

    def decrypt_one_round(self, c, k, rc=None):
        """
        Inverse round function of the cipher
        :param c: The ciphertext
        :param k: The round key
        :param rc: The round constant
        :return: The one round decryption of c using key k
        """
        p = [np.zeros(c[i].shape, dtype=self.word_dtype) for i in range(16)]
        # inverse permutation layer
        for i in range(4):
            p[4*i] = self.bit_to_pos(self.bit_at_pos(c[i], 3), 3) + self.bit_to_pos(self.bit_at_pos(c[4+i], 3), 2) + \
                     self.bit_to_pos(self.bit_at_pos(c[8+i], 3), 1) + self.bit_to_pos(self.bit_at_pos(c[12+i], 3), 0)

            p[4*i+1] = self.bit_to_pos(self.bit_at_pos(c[i], 2), 3) + self.bit_to_pos(self.bit_at_pos(c[4+i], 2), 2) + \
                       self.bit_to_pos(self.bit_at_pos(c[8+i], 2), 1) + self.bit_to_pos(self.bit_at_pos(c[12+i], 2), 0)

            p[4*i+2] = self.bit_to_pos(self.bit_at_pos(c[i], 1), 3) + self.bit_to_pos(self.bit_at_pos(c[4+i], 1), 2) + \
                       self.bit_to_pos(self.bit_at_pos(c[8+i], 1), 1) + self.bit_to_pos(self.bit_at_pos(c[12+i], 1), 0)

            p[4*i+3] = self.bit_to_pos(self.bit_at_pos(c[i], 0), 3) + self.bit_to_pos(self.bit_at_pos(c[4+i], 0), 2) + \
                       self.bit_to_pos(self.bit_at_pos(c[8+i], 0), 1) + self.bit_to_pos(self.bit_at_pos(c[12+i], 0), 0)
        for i in range(16):
            # inverse s-box layer
            p[i] = self.SBOXINV[p[i]]
            # add round key
            p[i] = p[i] ^ k[i]
        return p

    def decrypt(self, c, keys):
        """
        Decrypt by applying the inverse round function for each given key
        :param c: The ciphertext
        :param keys: A list of round keys
        :return: The decryption of c under the round keys in keys
        """
        c = c ^ keys[-1]
        return super(Present, self).decrypt(c, keys[:-1])

    def calc_back(self, c, p=None, variant=1):
        """
        Revert deterministic parts of the round function
        :param c: The ciphertext
        :param p: The initial plaintext
        :param variant: Select the variant of how to calculate back (default is 1; 0 means not calculating back)
        :return: The inner state after reverting the deterministic transformation at the end of the encryption process
        """
        if variant == 0:
            return c
        raise Exception("ERROR: No variant of calculating back is implemented")

    def key_schedule(self, key):
        """
        Applies the key schedule
        :param key: The key
        :return: A list of round keys
        """
        samples = len(key[0])
        ks = np.zeros((self.n_rounds, 20, samples), dtype=self.word_dtype)
        ks[0] = key.copy()
        for r in range(self.n_rounds - 1):
            lmk = ks[r].copy()
            for i in range(20):
                ks[r+1, i] = np.left_shift(lmk[(i+15) % 20] & 0b0111, 1) ^ np.right_shift(lmk[(i+16) % 20], 3)
            ks[r+1, 0] = self.SBOX[ks[r+1, 0]]
            ks[r+1, 15] = ks[r+1, 15] ^ ((r+1) >> 1)
            ks[r+1, 16] = ks[r+1, 16] ^ (((r+1) & 0b1) << 3)
        return ks[:, 0:16]

    @staticmethod
    def get_test_vectors():
        """
        Test vectors from https://link.springer.com/chapter/10.1007/978-3-540-74735-2_31, page 464 (15th page)
        :return: Returns the test vectors used for verifying the correct implementation of the cipher as a list of
            tuples of the form (cipher, plaintext, key, ciphertext), where cipher is an instance of self that will
            be used to verify the test vector
        """
        present_80 = Present()
        key = np.array([[0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf],
                        [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf],
                        [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf],
                        [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf],
                        [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf], [0, 0xf, 0, 0xf]], dtype=np.uint8)
        ks = present_80.key_schedule(key)
        pt = np.array([[0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf],
                       [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf],
                       [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf],
                       [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf], [0, 0, 0xf, 0xf]], dtype=np.uint8)
        ct = np.array([
            [0x5, 0xE, 0xA, 0x3], [0x5, 0x7, 0x1, 0x3], [0x7, 0x2, 0x1, 0x3], [0x9, 0xC, 0x2, 0x3],
            [0xC, 0x4, 0xF, 0xD], [0x1, 0x6, 0xF, 0xC], [0x3, 0xC, 0xC, 0xD], [0x8, 0x0, 0x7, 0x3],
            [0x7, 0xF, 0x2, 0x2], [0xB, 0x5, 0xF, 0x1], [0x2, 0x9, 0x6, 0x3], [0x2, 0x4, 0x8, 0x2],
            [0x8, 0x5, 0x4, 0x1], [0x4, 0x0, 0x1, 0x0], [0x4, 0x4, 0x7, 0xD], [0x5, 0x9, 0xB, 0x2]
        ], dtype=np.uint8)
        return [(present_80, pt, ks, ct)]


In [ ]:
import numpy as np

class Katan(AbstractCipher):

    def __init__(self, n_rounds=254, use_key_schedule=True):
        """
        Initializes a Katan cipher object
        :param n_rounds: The number of rounds used for de-/encryption
        :param use_key_schedule: Whether to use the key schedule or independent round keys
        """
        super(Katan, self).__init__(
            n_rounds, word_size=32, n_words=1,  # We see the state of katan32 as one word
            n_main_key_words=5, n_round_key_words=1, use_key_schedule=use_key_schedule,
            main_key_word_size=16, round_key_word_size=2
        )
        self.len_l1 = 13
        self.len_l2 = 19

    @staticmethod
    def get_name():
        return "Katan"

    IR = (1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
          1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
          0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
          0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
          1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
          0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
          1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
          1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
          1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
          0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
          1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
          0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
          1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0)

    @staticmethod
    def parse_as_bits(x, word_size=32):
        """
        Parses each element of p as a list of bits with the lsb of each element at position i = 0
        :param x: The current state
        :param word_size: The word size of x
        :return: List of bits representing p
        """
        bits = np.zeros(np.insert(x.shape, 0, word_size), dtype=np.uint8)
        for i in range(word_size):
            bits[i] = np.right_shift(x, i) & 0b1
        return bits

    def parse_as_int(self, bits):
        """
        Parses a list of bits to the corresponding integers
        :param bits: The bits
        :return: Integers representing bits
        """
        num = np.zeros(bits.shape[1:], dtype=self.word_dtype)
        for i in range(bits.shape[0]-1, -1, -1):
            num = np.left_shift(num, 1) ^ bits[i]
        return num

    def encrypt_one_round(self, p, k, rc=None):
        """
        Round function of the cipher
        :param p: The plaintext
        :param k: The round key
        :param rc: The round constant
        :return: The one round encryption of p using key k
        """
        k_a = k & 0x1
        k_b = (k >> 1) & 0x1

        f_a = p[self.len_l2 + 12] ^ p[self.len_l2 + 7] ^ (p[self.len_l2 + 8] & p[self.len_l2 + 5]) ^ k_a
        if rc:
            f_a = f_a ^ p[self.len_l2 + 3]

        f_b = p[18] ^ p[7] ^ (p[12] & p[10]) ^ (p[8] & p[3]) ^ k_b

        # Shift L1 and L2 by 1
        p = np.roll(p, 1, axis=0)
        # Set value of feedback function
        p[0] = f_a
        p[self.len_l2] = f_b
        return p

    def encrypt(self, p, keys):
        """
        Encrypt by applying the round function for each given round key
        :param p: The plaintext
        :param keys: A list of round keys
        :return: The encryption of p under the round keys in keys
        """
        state = self.parse_as_bits(p)
        c = super(Katan, self).encrypt(state, keys)
        return self.parse_as_int(c)

    def decrypt_one_round(self, c, k, rc=None):
        """
        Inverse round function of the cipher
        :param c: The ciphertext
        :param k: The round key
        :param rc: The round constant
        :return: The one round decryption of c using key k
        """
        k_a = k & 0x1
        k_b = (k >> 1) & 0x1

        f_a = c[0] ^ c[self.len_l2 + 8] ^ (c[self.len_l2 + 9] & c[self.len_l2 + 6]) ^ k_a
        if rc:
            f_a = f_a ^ c[self.len_l2 + 4]

        f_b = c[self.len_l2 + 0] ^ c[8] ^ (c[13] & c[11]) ^ (c[9] & c[4]) ^ k_b

        # Shift L1 and L2 by 1
        c = np.roll(c, -1, axis=0)
        # Set value of feedback function
        c[self.len_l2 - 1] = f_b  # L2[-1] = f_b
        c[-1] = f_a  # L1[-1] = f_a
        return c

    def decrypt(self, c, keys):
        """
        Decrypt by applying the inverse round function for each given key
        :param c: The ciphertext
        :param keys: A list of round keys
        :return: The decryption of c under the round keys in keys
        """
        state = self.parse_as_bits(c)
        p = super(Katan, self).decrypt(state, keys)
        return self.parse_as_int(p)

    def calc_back(self, c, p=None, variant=1):
        """
        Revert deterministic parts of the round function
        :param c: The ciphertext
        :param p: The initial plaintext
        :param variant: Select the variant of how to calculate back (default is 1; 0 means not calculating back)
            Options are
                - 1: Only revert 4 full rounds (except for key addition)
                - 2: Revert 17 rounds as far as possible
        :return: The inner state after reverting the deterministic transformation at the end of the encryption process
        """
        if variant == 0:
            return c
        c = self.parse_as_bits(c)
        # Calculate 4 rounds back with keys set to zero
        for i in range(4):
            c = self.decrypt_one_round(c, 0, self.get_rc(self.n_rounds - 1 - i))
        if variant == 1:  # Revert 4 rounds except of key addition
            return self.parse_as_int(c)
        if variant == 2:  # Revert 17 rounds as far as possible
            for i in range(13):  # Note: We already reverted 4 rounds
                f_a = c[0]
                f_b = c[self.len_l2 + 0]
                # For all rounds, L1[x3]&L1[x4] is not known
                # For i >= 2, L2[y3]&L2[y4] is not known
                # For i >= 5,L1[8] is also masked by L1[x3]&L1[x4], with L1[x3] masked by a key bit
                # Also, L1[x5] is now masked by a key bit
                # For i >= 6, L2[y5]&L2[y6] is not known
                if i < 2:
                    f_b = f_b ^ (c[13] & c[11])
                if i < 5:
                    f_a = f_a ^ c[self.len_l2 + 8]  # Note that L1[8] may be masked with a key bit
                    if self.get_rc(self.n_rounds - 5 - i):
                        f_a = f_a ^ c[self.len_l2 + 4]
                if i < 6:
                    f_b = f_b ^ (c[9] & c[4])
                f_b = f_b ^ c[8]
                # Shift L1 and L2 by 1
                c = np.roll(c, -1, axis=0)
                # Set value of feedback function
                c[self.len_l2 - 1] = f_b  # L2[-1] = f_b
                c[-1] = f_a  # L1[-1] = f_a
            return self.parse_as_int(c)
        raise Exception(f'ERROR: Variant {variant} of calculating back is not implemented')

    def get_rc(self, r):
        """
        :param r: The round
        :return: The round constant for round r
        """
        return self.IR[r]

    def key_schedule(self, key):
        """
        Applies the key schedule
        :param key: The key
        :return: A list of round keys
        """
        # Parse masterKey as 80 bit vector
        samples = key.shape[1]
        state = np.concatenate([self.parse_as_bits(key[i], 16) for i in range(key.shape[0])], axis=0)
        key_bits = state.shape[0]

        ks = np.zeros((self.n_rounds, samples), dtype=np.uint8)
        for i in range(self.n_rounds):
            k = 0  # First bit (lsb) will be k_a, second k_b
            for j in range(2):
                zero_pos = (2 * i + j) % key_bits
                k ^= state[zero_pos] << j
                h = state[zero_pos + 0] ^ state[(zero_pos + 19) % key_bits] ^ \
                    state[(zero_pos + 30) % key_bits] ^ state[(zero_pos + 67) % key_bits]
                state[zero_pos] = h
            ks[i] = k
        return ks

    @staticmethod
    def get_test_vectors():
        """
        Test vectors from http://www.cs.technion.ac.il/~orrd/KATAN/
        :return: Returns the test vectors used for verifying the correct implementation of the cipher as a list of
            tuples of the form (cipher, plaintext, key, ciphertext), where cipher is an instance of self that will
            be used to verify the test vector
        """
        katan32 = Katan()
        key = np.array([[0xffff,     0],
                        [0xffff,     0],
                        [0xffff,     0],
                        [0xffff,     0],
                        [0xffff,     0]], dtype=np.uint16)
        ks = katan32.key_schedule(key)
        pt = np.array([[0, 0xffffffff]], dtype=np.uint32)
        ct = np.array([[0x7E1FF945, 0x432E61DA]], dtype=np.uint32)
        return [(katan32, pt, ks, ct)]

In [ ]:
import numpy as np
from os import urandom



class ChaCha(AbstractCipher):

    def __init__(self, n_rounds=20, use_key_schedule=True):
        """
        Initializes a ChaCha cipher object
        :param n_rounds: The number of rounds used for de-/encryption
        :param use_key_schedule: Whether to use the key schedule or independent round keys
        """
        if not use_key_schedule:
            raise Exception('ERROR: use_key_schedule=False not supported for ChaCha')
        super(ChaCha, self).__init__(
            n_rounds, word_size=32, n_words=16, n_main_key_words=8,
            n_round_key_words=None, use_key_schedule=use_key_schedule
        )


    @staticmethod
    def get_name():
        return "ChaCha"

    def quarter_round(self, state, a, b, c, d):
        """
        Applies the ChaCha quarter round to state
        :param state: The state
        :param a: Index of input a
        :param b: Index of input b
        :param c: Index of input c
        :param d: Index of input d
        :return: The manipulated state
        """
        state[a] += state[b]
        state[d] ^= state[a]
        state[d] = self.rol(state[d], 16)
        state[c] += state[d]
        state[b] ^= state[c]
        state[b] = self.rol(state[b], 12)
        state[a] += state[b]
        state[d] ^= state[a]
        state[d] = self.rol(state[d], 8)
        state[c] += state[d]
        state[b] ^= state[c]
        state[b] = self.rol(state[b], 7)
        return state

    def encrypt_one_round(self, p, k, rc=None):
        """
        Round function of the cipher
        :param p: The plaintext
        :param k: The round key
        :param rc: The round constant
        :return: The one round encryption of p using key k
        """
        s = p  # p is the state which already contains the key. Hence, k is not used
        if rc == 0:  # This is a column round
            s = self.quarter_round(s, 0, 4, 8, 12)
            s = self.quarter_round(s, 1, 5, 9, 13)
            s = self.quarter_round(s, 2, 6, 10, 14)
            s = self.quarter_round(s, 3, 7, 11, 15)
        else:  # This is a diagonal round
            s = self.quarter_round(s, 0, 5, 10, 15)
            s = self.quarter_round(s, 1, 6, 11, 12)
            s = self.quarter_round(s, 2, 7, 8, 13)
            s = self.quarter_round(s, 3, 4, 9, 14)
        return s

    @staticmethod
    def build_initial_state(nonce_and_counter, key):
        """
        :param nonce_and_counter: Nonce and Counter
        :param key: The key
        :return: The initial state
        """
        n_samples = len(nonce_and_counter[0])
        constants = np.array([0x61707865, 0x3320646e, 0x79622d32, 0x6b206574], dtype=np.uint32)
        constants = np.repeat(np.expand_dims(constants, axis=1), n_samples, axis=1)  # Repeat for every sample
        return np.concatenate((constants, key, nonce_and_counter), axis=0)

    def encrypt(self, p, keys):
        """
        Generate the key stream. Note that, since this framework was designed with block ciphers in mind,
        the usage of the parameters is slightly different.
        :param p: The data an attacker is able to control, i.e. the counter and the nonce
        :param keys: The key that is used to generate the key stream block
        :return: The key stream block
        """
        state = self.build_initial_state(p, keys)
        working_state = state.copy()

        # Iterate rounds
        working_state = super(ChaCha, self).encrypt(working_state, [None for i in range(self.n_rounds)])

        # Calculate final key stream block and return
        return state + working_state

    def decrypt_one_round(self, c, k, rc=None):
        raise NotImplementedException("decrypt", "ChaCha")

    def calc_back(self, c, p=None, variant=1):
        """
        Revert deterministic parts of the round function
        :param c: The ciphertext
        :param p: The initial plaintext
        :param variant: Select the variant of how to calculate back (default is 1; 0 means not calculating back)
        :return: The inner state after reverting the deterministic transformation at the end of the encryption process
        """
        if variant == 0:
            return c
        if variant != 1:
            raise Exception(f'ERROR: Variant {variant} of calculating back is not implemented')
        if p is None:
            raise Exception("ERROR: Nonce and counter are needed in order to calculate state back")
        # Build known initial state
        zero_key = np.zeros((8, len(p[0])), dtype=np.uint32)
        known_initial_state = self.build_initial_state(p, zero_key)
        # Revert addition of initial state to working state for counter, constant and nonce
        return c - known_initial_state

    def get_rc(self, r):
        """
        :param r: The round
        :return: The round constant for round r
        """
        return r % 2  # To distinguish between column and diagonal rounds

    def draw_plaintexts(self, n_samples):
        """
        Draw nonce and counter, i.e. 4 words
        :param n_samples: How many plaintexts to draw
        :return: An array of plaintexts
        """
        return np.reshape(
            np.frombuffer(urandom(4 * 4 * n_samples), dtype=self.word_dtype),
            (4, n_samples)
        )

    def key_schedule(self, key):
        """
        Applies the key schedule
        :param key: The key
        :return: A list of round keys
        """
        return key  # There exist no round keys fo ChaCha and the main key has already the shape we need

    @staticmethod
    def get_test_vectors():
        """
        Test vectors from https://tools.ietf.org/pdf/rfc7539.pdf, page 9
        :return: Returns the test vectors used for verifying the correct implementation of the cipher as a list of
            tuples of the form (cipher, plaintext, key, ciphertext), where cipher is an instance of self that will
            be used to verify the test vector
        """
        chacha = ChaCha()
        key = np.array([[0x03020100], [0x07060504], [0x0b0a0908], [0x0f0e0d0c],
                        [0x13121110], [0x17161514], [0x1b1a1918], [0x1f1e1d1c]], dtype=np.uint32)
        nonce_and_counter = np.array([[0x00000001], [0x09000000], [0x4a000000], [0x00000000]], dtype=np.uint32)
        stream = np.array([
            [0xe4e7f110], [0x15593bd1], [0x1fdd0f50], [0xc47120a3],
            [0xc7f4d1c7], [0x0368c033], [0x9aaa2204], [0x4e6cd4c3],
            [0x466482d2], [0x09aa9f07], [0x05d7c214], [0xa2028bd9],
            [0xd19c12b5], [0xb94e16de], [0xe883d0cb], [0x4e3c50a2]
        ], dtype=np.uint32)
        return [(chacha, nonce_and_counter, key, stream)]

## Generate datasets

Based on code from: https://github.com/differential-neural/An-Assessment-of-Differential-Neural-Distinguishers

In [ ]:
from os import urandom
import numpy as np

def convert_to_binary(arr, n_words, word_size, n_pairs = 1): #-> np.ndarray
    if n_words == 0:
       sample_len =  word_size
    else:
      sample_len = 2 * n_words * word_size * n_pairs

    n_samples = len(arr[0])

    x = np.zeros((sample_len, n_samples), dtype=np.uint8)

    for i in range(sample_len):
        index = i // word_size
        offset = word_size - (i % word_size) - 1
        x[i] = (arr[index] >> offset) & 1
    x = x.transpose()

    return x


def preprocess_samples(ct0, ct1, pt0, pt1, cipher, calc_back=0, data_format=None): #-> np.ndarray:
    if data_format:
        if calc_back != 0:
            raise Exception("Backwards calculation is not compatible with using a specific data format!")

        if data_format == "Diffs":
          diffs = []
          for block in range(0, cipher.get_n_words()):
              diff = ct0[block] ^ ct1[block]
              arr  = [np.array(diff, dtype=cipher.main_key_word_dtype)]
              diffs.append(arr)

          concat = np.concatenate((tuple(diffs)), axis=0)

          return convert_to_binary(concat, int(cipher.get_n_words()/2), cipher.get_word_size())

        if data_format == "Pairs":
          return convert_to_binary(np.concatenate((ct0, ct1), axis=0), cipher.get_n_words(), cipher.get_word_size())


        if data_format == "LLS+23" and type(cipher) is Simon:
            # calculate diff = (\Delta_L^r,\Delta_R^r)
            diff_l = ct0[0] ^ ct1[0]
            diff_r = ct0[1] ^ ct1[1]
            # calculate diff_r_previous = \Delta_R^{r-1}
            previous0 = cipher.calc_back(ct0, pt0, 1)
            previous1 = cipher.calc_back(ct1, pt1, 1)
            diff_r_previous = previous0[1] ^ previous1[1]
            # calculate diff_r_penultimate = p\Delta_R^{r-2}
            penultimate0 = cipher.calc_back(previous0, pt0, 1)
            penultimate1 = cipher.calc_back(previous1, pt1, 1)
            diff_r_penultimate = penultimate0[1] ^ penultimate1[1]
            # merge into (\Delta_L^r, \Delta_R^r, C_l, C_r, C_l', C_r', \Delta_R^{r-1}, p\Delta_R^{r-2})
            return convert_to_binary(np.concatenate(((diff_l, diff_r), ct0, ct1, (diff_r_previous, diff_r_penultimate)), axis=0), 4, cipher.get_word_size())


        if data_format == "PairsPlus":
            previous0 = cipher.calc_back(ct0, pt0, 1)
            previous1 = cipher.calc_back(ct1, pt1, 1)
            return convert_to_binary(np.concatenate((previous0, previous1, ct0, ct1), axis=0), cipher.get_n_words(), cipher.get_word_size(), n_pairs = 2)

    if calc_back != 0:
        # note that the plaintext gets used for ChaCha only
        ct0 = cipher.calc_back(ct0, pt0, calc_back)
        ct1 = cipher.calc_back(ct1, pt1, calc_back)
    # convert to binary and return

    return convert_to_binary(np.concatenate((ct0, ct1), axis=0), cipher.get_n_words(), cipher.get_word_size())


def make_train_data(n_samples, cipher, diff, calc_back=0, y=None, additional_conditions=None, data_format=None): #-> (np.ndarray, np.ndarray):

    # generate labels
    if y is None:
        y = np.frombuffer(urandom(n_samples), dtype=np.uint8) & 1
    elif y == 0 or y == 1:
        y = np.array([y for _ in range(n_samples)], dtype=np.uint8)
    # draw keys and plaintexts
    keys = cipher.draw_keys(n_samples)
    pt0 = cipher.draw_plaintexts(n_samples)
    if additional_conditions is not None:
        pt0 = additional_conditions(pt0)
    pt1 = pt0 ^ np.array(diff, dtype=cipher.word_dtype)[:, np.newaxis]
    # replace plaintexts in pt1 with random ones if label is 0
    num_rand_samples = np.sum(y == 0)
    pt1[:, y == 0] = cipher.draw_plaintexts(num_rand_samples)
    # encrypt
    ct0 = cipher.encrypt(pt0, keys)
    ct1 = cipher.encrypt(pt1, keys)
    # perform backwards calculation and other preprocessing
    x = preprocess_samples(ct0, ct1, pt0, pt1, cipher, calc_back, data_format)
    return x, y



def make_real_differences_data(n_samples, cipher, diff, calc_back=0): #-> (np.ndarray, np.ndarray):

    # generate labels
    y = np.frombuffer(urandom(n_samples), dtype=np.uint8) & 1
    # draw keys and plaintexts
    keys = cipher.draw_keys(n_samples)
    pt0 = cipher.draw_plaintexts(n_samples)
    pt1 = pt0.copy() ^ np.array(diff, dtype=cipher.word_dtype)[:, np.newaxis]
    # encrypt
    ct0 = cipher.encrypt(pt0, keys)
    ct1 = cipher.encrypt(pt1, keys)
    if calc_back != 0:
        ct0 = cipher.calc_back(ct0, pt0, calc_back)
        ct1 = cipher.calc_back(ct1, pt1, calc_back)
    if type(ct0) is tuple or type(ct0) is list:  # Convert ciphertexts into numpy arrays in case they aren't already
        ct0 = np.array(ct0, dtype=cipher.word_dtype)
        ct1 = np.array(ct1, dtype=cipher.word_dtype)
    # blind samples with label 0
    num_rand_samples = np.sum(y == 0)
    blinding_val = cipher.draw_ciphertexts(num_rand_samples)
    ct0[:, y == 0] ^= blinding_val
    ct1[:, y == 0] ^= blinding_val
    # convert to binary and return
    x = convert_to_binary(np.concatenate((ct0, ct1), axis=0), cipher.get_n_words(), cipher.get_word_size())
    return x, y


## Train

Based on code from:
1. https://github.com/differential-neural/An-Assessment-of-Differential-Neural-Distinguishers
2. https://github.com/CryptAnalystDesigner/NeuralDistingsuisherWithInception
3. https://github.com/Crypto-TII/AutoND

In [ ]:
import numpy as np
from pickle import dump
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, Lambda, concatenate
from tensorflow.keras.regularizers import l2




bs = 5000
wdir = './freshly_trained_nets/'


def cyclic_lr(num_epochs, high_lr, low_lr):
    return lambda i: low_lr + ((num_epochs - 1) - i % num_epochs) / (num_epochs - 1) * (high_lr - low_lr)

class MyLRSchedule(keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, initial_learning_rate, final_learning_rate, num_epochs):
        self.initial_learning_rate = initial_learning_rate
        self.final_learning_rate = final_learning_rate
        self.num_epochs = num_epochs

    def __call__(self, step):
        return self.final_learning_rate + ((self.num_epochs - 1) - step % self.num_epochs) / (self.num_epochs - 1) * (self.initial_learning_rate - self.final_learning_rate)



def make_resnet(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):

    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)

    train_shape = None
    search_ks = None
    ks1 = 1
    ks = 3

    if shape == "UNSHAPED":
      train_shape = Reshape((num_blocks * word_size * 2, 1))(inp)
      search_ks = num_blocks * word_size
    else:
      train_shape = perm
      search_ks = word_size





    if tune is True:
      reg_param = hp.Float("reg_param", 0.000000001 , 0.0001, step=0.000000001, default = reg_param)
      #ks1 = hp.Int('ks1', min_value=1, max_value= search_ks, step=1, default = ks1)
      ks = hp.Int('ks', min_value=1, max_value= search_ks, step=1, default = ks)
      high_lr = hp.Float("high_lr", 0.0001 , 0.01, step=0.0001, default = high_lr)
      low_lr = hp.Float("low_lr", 0.000001 , 0.001, step=0.000001, default = low_lr)





    conv0 = Conv1D(32, kernel_size= ks1, padding='same', kernel_regularizer=l2(reg_param))(train_shape)
    conv0 = BatchNormalization()(conv0)
    conv0 = Activation('relu')(conv0)

    shortcut = conv0
    for i in range(10):
        conv1 = Conv1D(32, kernel_size= ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation('relu')(conv1)
        conv2 = Conv1D(32, kernel_size= ks, padding='same', kernel_regularizer=l2(reg_param))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)
        shortcut = Add()([shortcut, conv2])

    flat1 = Flatten()(shortcut)

    dense1 = Dense(64, kernel_regularizer=l2(reg_param))(flat1)
    dense1 = BatchNormalization()(dense1)
    dense1 = Activation('relu')(dense1)

    dense2 = Dense(64, kernel_regularizer=l2(reg_param))(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Activation('relu')(dense2)

    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(dense2)
    model = Model(inputs=inp, outputs=out)

    #Add this everywhere and remove from main
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = MyLRSchedule(high_lr, low_lr, 5)), loss='mse', metrics=['acc'])

    return model



def make_MLP(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):

    chunks = 0

    #Add variant for False with INSHAPE
    special_treatment_when_input_is_INSHAPE = True

    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
      chunks = num_blocks
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
      chunks = num_blocks * 2
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)
      chunks = num_blocks * 4


    if tune is True:
      reg_param = hp.Float("reg_param", 0.000000001 , 0.0001, step=0.000000001, default = reg_param)
      high_lr = hp.Float("high_lr", 0.0001 , 0.01, step=0.0001, default = high_lr)
      low_lr = hp.Float("low_lr", 0.000001 , 0.001, step=0.000001, default = low_lr)




    outputs = []
    combined = None





    if shape == "UNSHAPED":

      dense = Dense(chunks * word_size, kernel_regularizer=l2(reg_param))(inp)
      dense = BatchNormalization()(dense)
      dense = Activation('relu')(dense)

      dense = Dense(chunks * word_size, kernel_regularizer=l2(reg_param))(dense)
      dense = BatchNormalization()(dense)
      dense = Activation('relu')(dense)

      combined = dense

    else: #For shape a simple MLP that takes reshaped input does not work - but when given the full unshaped input a simple MLP does
          #To boost the performance we do this special thing to train on reshaped input

      if special_treatment_when_input_is_INSHAPE is True:
          for chunk in list(range(0,chunks)):
              dense = Dense(word_size, kernel_regularizer=l2(reg_param))(perm[:, :, chunk])
              dense = BatchNormalization()(dense)
              dense = Activation('relu')(dense)

              dense = Dense(word_size, kernel_regularizer=l2(reg_param))(dense)
              dense = BatchNormalization()(dense)
              dense = Activation('relu')(dense)
              outputs.append(dense)

          # Concatenate the outputs of the four MLPs
          combined = keras.layers.concatenate(outputs)
      else:
        dense = Dense(word_size, kernel_regularizer=l2(reg_param))(perm)
        dense = BatchNormalization()(dense)
        dense = Activation('relu')(dense)

        dense = Dense(word_size, kernel_regularizer=l2(reg_param))(dense)
        dense = BatchNormalization()(dense)
        dense = Activation('relu')(dense)

        combined = dense



    num_layers_init =  3
    #size = [64, 64, 32]
    size_init = [chunks * word_size, chunks * word_size, int(chunks * word_size/2)]

    if tune is True:
      num_layers = hp.Int('num_layers', min_value=1, max_value= chunks*word_size, step=1, default = num_layers_init)

    x = combined
    counter = 0
    for i in range(0, num_layers):
      if tune is True:
        size = hp.Int('size_layer'+str(counter), min_value=1, max_value= chunks*word_size * 4, step=1)
        x = Dense(size, kernel_regularizer=l2(reg_param))(x)
      else:
        size = size_init
        x = Dense(size[counter], kernel_regularizer=l2(reg_param))(x)


      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      counter = counter + 1

    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(x)
    model = Model(inputs=inp, outputs=out)

    model.compile(optimizer= keras.optimizers.Adam(learning_rate = MyLRSchedule(high_lr, low_lr, 5)), loss='mse', metrics=['acc'])


    return model



def make_Gohr_Inception_1stB(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):

    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)

    train_shape = None
    search_ks = None
    num_filters=32

    ks1 = 1
    ks2 = 3
    ks3 = 5
    ks4 = 7
    ks = 3

    if shape == "UNSHAPED":
      train_shape = Reshape((num_blocks * word_size * 2, 1))(inp)
      search_ks = num_blocks * word_size
    else:
      train_shape = perm
      search_ks = word_size




    if tune is True:
      reg_param = hp.Float("reg_param", 0.000000001 , 0.0001, step=0.000000001, default = reg_param)
      #ks1 = hp.Int('ks1', min_value=1, max_value= search_ks, step=1, default = ks1)
      ks2 = hp.Int('ks2', min_value=1, max_value= search_ks, step=1, default = ks2)
      ks3 = hp.Int('ks3', min_value=1, max_value= search_ks, step=1, default = ks3)
      ks4 = hp.Int('ks4', min_value=1, max_value= search_ks, step=1, default = ks4)
      ks = hp.Int('ks', min_value=1, max_value= search_ks, step=1, default = ks)
      high_lr = hp.Float("high_lr", 0.0001 , 0.01, step=0.0001, default = high_lr)
      low_lr = hp.Float("low_lr", 0.000001 , 0.001, step=0.000001, default = low_lr)

    conv01 = Conv1D(num_filters, kernel_size=ks1, padding='same', kernel_regularizer=l2(reg_param))(perm)
    conv02 = Conv1D(num_filters, kernel_size=ks2, padding='same', kernel_regularizer=l2(reg_param))(perm)
    conv03 = Conv1D(num_filters, kernel_size=ks3, padding='same', kernel_regularizer=l2(reg_param))(perm)
    conv04 = Conv1D(num_filters, kernel_size=ks4, padding='same', kernel_regularizer=l2(reg_param))(perm)

    c2 = keras.layers.concatenate([conv01, conv02, conv03, conv04], axis=-1)
    conv0 = BatchNormalization()(c2)
    conv0 = Activation('relu')(conv0)

    shortcut = conv0


    for i in range(10):
        conv1 = Conv1D(num_filters*4, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation('relu')(conv1)
        conv2 = Conv1D(num_filters*4, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)
        shortcut = Add()([shortcut, conv2])
        ks += 2


    flat1 = Flatten()(shortcut)
    dense0 = keras.layers.Dropout(0.8)(flat1) #Dropout added to reduce overfitting - as mentioned in paper
    dense0 = Dense(512, kernel_regularizer=l2(reg_param))(dense0)

    # dense0 = Dense(512, kernel_regularizer=l2(reg_param))(flat1)

    dense0 = BatchNormalization()(dense0)
    dense0 = Activation('relu')(dense0)
    dense1 = Dense(64, kernel_regularizer=l2(reg_param))(dense0)
    dense1 = BatchNormalization()(dense1)
    dense1 = Activation('relu')(dense1)
    dense2 = Dense(64, kernel_regularizer=l2(reg_param))(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Activation('relu')(dense2)
    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(dense2)
    model = Model(inputs=inp, outputs=out)

    model.compile(optimizer= keras.optimizers.Adam(learning_rate = MyLRSchedule(high_lr, low_lr, 5)), loss='mse', metrics=['acc'])


    return(model)




def make_Gohr_Inception_2ndB(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):

    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)

    train_shape = None
    search_ks = None
    num_filters=32
    ks1 = 1
    ks01 = 1
    ks02 = 2
    ks03 = 4
    ks04 = 8
    ks2 = 1


    if shape == "UNSHAPED":
      train_shape = Reshape((num_blocks * word_size * 2, 1))(inp)
      search_ks = num_blocks * word_size
    else:
      train_shape = perm
      search_ks = word_size


    if tune is True:
      reg_param = hp.Float("reg_param", 0.000000001 , 0.0001, step=0.000000001, default = reg_param)
      #ks1 = hp.Int('ks1', min_value=1, max_value= search_ks, step=1, default = ks1)
      ks01 = hp.Int('ks01', min_value=1, max_value= search_ks, step=1, default = ks01)
      ks02 = hp.Int('ks02', min_value=1, max_value= search_ks, step=1, default = ks02)
      ks03 = hp.Int('ks03', min_value=1, max_value= search_ks, step=1, default = ks03)
      ks04 = hp.Int('ks04', min_value=1, max_value= search_ks, step=1, default = ks04)
      #ks2 = hp.Int('ks2', min_value=1, max_value= search_ks, step=1, default = ks2)

      high_lr = hp.Float("high_lr", 0.0001 , 0.01, step=0.0001, default = high_lr)
      low_lr = hp.Float("low_lr", 0.000001 , 0.001, step=0.000001, default = low_lr)

    conv0 = Conv1D(3 * num_filters, kernel_size=ks1, padding='same', kernel_regularizer=l2(reg_param))(train_shape)
    conv0 = BatchNormalization()(conv0)
    conv0 = Activation('relu')(conv0)

    shortcut = conv0
    for i in range(10):
        conv = Conv1D(3 * num_filters, kernel_size=ks01, padding='same', kernel_regularizer=l2(reg_param))(shortcut)

        conv01 = Conv1D(num_filters, kernel_size=ks02, padding='same', kernel_regularizer=l2(reg_param))(conv)
        conv02 = Conv1D(num_filters, kernel_size=ks03, padding='same', kernel_regularizer=l2(reg_param))(conv)
        conv03 = Conv1D(num_filters, kernel_size=ks04, padding='same', kernel_regularizer=l2(reg_param))(conv)

        concat = keras.layers.concatenate([conv01, conv02, conv03], axis=-1)
        concat = BatchNormalization()(concat)
        concat = Activation('relu')(concat)

        conv2 = Conv1D(3 * num_filters, kernel_size=ks2, padding='same', kernel_regularizer=l2(reg_param))(concat)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)
        shortcut = Add()([shortcut, conv2])

    flat = Flatten()(shortcut)

    #pool = keras.layers.GlobalAveragePooling1D()(flat)

    dense1 = Dense(64, kernel_regularizer=l2(reg_param))(flat)
    dense1 = BatchNormalization()(dense1)
    dense1 = Activation('relu')(dense1)

    dense2 = Dense(64, kernel_regularizer=l2(reg_param))(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Activation('relu')(dense2)

    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(dense2)
    model = Model(inputs=inp, outputs=out)

    model.compile(optimizer= keras.optimizers.Adam(learning_rate = MyLRSchedule(high_lr, low_lr, 5)), loss='mse', metrics=['acc'])


    return model




def get_dilation_rates(input_size):
    """Helper function to determine the dilation rates of DBitNet given an input_size. """
    drs = []
    while input_size >= 8:
        drs.append(int(input_size / 2 - 1))
        input_size = input_size // 2

    return drs


def make_DBIT(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):
    input_size = None

    #From here, only the input size is taken as the model does not need reshaping
    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
      input_size= num_blocks * word_size * 1
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
      input_size= num_blocks * word_size * 2
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)
      input_size= num_blocks * word_size * 4


    '''
    train_shape = None
    if shape == "UNSHAPED":
      train_shape = Reshape((num_blocks * word_size * 2, 1))(inp)
    else:
      train_shape = perm
    '''

    n_filters=32
    n_add_filters=16


    # determine the dilation rates from the given input size
    dilation_rates = get_dilation_rates(input_size)

    # prediction head parameters (similar to Gohr)
    d1 = 256 # TODO this can likely be reduced to 64.
    d2 = 64
    reg_param = 1e-5

    # define the input shape
    inputs = Input(shape=(input_size, 1))
    x = inputs

    # normalize the input data to a range of [-1, 1]:
    x = tf.subtract(x, 0.5)
    x = tf.divide(x, 0.5)

    for dilation_rate in dilation_rates:
        ### wide-narrow blocks
        x = Conv1D(filters=n_filters,kernel_size=2,padding='valid', dilation_rate=dilation_rate,strides=1,activation='relu')(x)
        x = BatchNormalization()(x)
        x_skip = x
        x = Conv1D(filters=n_filters,kernel_size=2,padding='causal',dilation_rate=1,activation='relu')(x)
        x = Add()([x, x_skip])
        x = BatchNormalization()(x)

        n_filters += n_add_filters

    ### prediction head
    out = tf.keras.layers.Flatten()(x)

    dense0 = Dense(d1, kernel_regularizer=l2(reg_param))(out);
    dense0 = BatchNormalization()(dense0);
    dense0 = Activation('relu')(dense0);
    dense1 = Dense(d1, kernel_regularizer=l2(reg_param))(dense0);
    dense1 = BatchNormalization()(dense1);
    dense1 = Activation('relu')(dense1);
    dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1);
    dense2 = BatchNormalization()(dense2);
    dense2 = Activation('relu')(dense2);
    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(dense2)

    model = Model(inputs, out)

    model.compile(optimizer= tf.keras.optimizers.Adam(amsgrad=True), loss='mse', metrics=['acc'])


    return model





#def create_distinguisher(cipher, data_format, network, shape, tune, num_epochs, hp):
def create_distinguisher(hp):
    high_lr=0.002
    low_lr=0.0001
    reg_param=0.0001


    # create the network
    net = None

    if G_network == "Gohr":
        net = make_resnet(hp = hp, tune = G_tune, data_format = G_data_format, shape = G_shape, num_blocks = G_cipher.get_n_words(), word_size = G_cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = G_num_epochs)

    if G_network == "MLP": #TD
        net = make_MLP(hp = hp, tune = G_tune, data_format = G_data_format, shape = G_shape, num_blocks = G_cipher.get_n_words(), word_size = G_cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = G_num_epochs)

    if G_network == "GohrInception1stB":

        net = make_Gohr_Inception_1stB(hp = hp, tune = G_tune, data_format = G_data_format, shape = G_shape, num_blocks = G_cipher.get_n_words(), word_size = G_cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = G_num_epochs)

    if G_network == "GohrInception2ndB":
        net = make_Gohr_Inception_2ndB(hp = hp, tune = G_tune, data_format = G_data_format, shape = G_shape, num_blocks = G_cipher.get_n_words(), word_size = G_cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = G_num_epochs)

    if G_network == "DBIT":
        net = make_DBIT(hp = hp, tune = G_tune, data_format = G_data_format, shape = G_shape, num_blocks = G_cipher.get_n_words(), word_size = G_cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = G_num_epochs)


    return net

## Evaluate

Based on code from: https://github.com/agohr/deep_speck

In [ ]:
import numpy as np

def evaluate(net, X, Y):
    Z = net.predict(X, batch_size=10000).flatten()
    Zbin = (Z > 0.5)

    n = len(Z)
    n0 = np.sum(Y == 0)
    n1 = np.sum(Y == 1)

    acc = np.sum(Zbin == Y) / n
    tpr = np.sum(Zbin[Y == 1]) / n1
    tnr = np.sum(Zbin[Y == 0] == 0) / n0

    print("Accuracy: ", acc, "TPR: ", tpr, "TNR: ", tnr)
    return (acc, tpr, tnr)

## Train and evaluate

In [ ]:
def train_and_evaluate(experiment_name, cipher_class, n_rounds, diff, reps, data_format, network = "Gohr", shape = "INSHAPE", tune = False, num_epochs = 10):


  cipher = cipher_class(n_rounds)

  print("SET GLOBAL VARIABLES")
  set_global_variables(cipher = cipher, data_format = data_format, network = network, shape = shape, tune = tune, num_epochs = num_epochs)
  print_global_variables()
  print("Blocks: " + str(cipher.get_n_words()))
  print("Word size: " + str(cipher.get_word_size()))

  n_train_samples=10**7
  n_val_samples=10**6
  n_test_samples = 10**6

  batch_size = 5000


  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(n_rounds) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + "Tuning:" + str(tune) +".txt", "a")
  f.write("Cipher: " + str(cipher_class.get_name()) + " - " + str(n_rounds) +" Rounds" + " - Blocks: " + str(cipher.get_n_words()) + " - Word size: "+ str(cipher.get_word_size()) + "\n")
  f.write("Input difference: " + str(diff) + "\n")
  f.write("Network: " + network + " - Tuning: " + str(tune) + "\n")
  f.write("Trains on: " + data_format + " - " + shape + "\n")
  f.close()



  best_model = None



  if tune is True:
      tuner = keras_tuner.BayesianOptimization(create_distinguisher,
                                              objective='val_acc',
                                              max_trials=50)

      x_train, y_train = make_train_data(n_samples = n_train_samples, cipher = cipher, diff = diff, data_format = data_format)
      x_val, y_val     = make_train_data(n_samples = n_val_samples,   cipher = cipher, diff = diff, data_format = data_format)



      stop_early_tune = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience= 3, restore_best_weights= True)
      tuner.search(x_train, y_train, epochs = 10, batch_size=batch_size, validation_data=(x_val, y_val), callbacks=[stop_early_tune])
      best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

      print("------------------------------------\n")
      print("TUNER SUMMARY:")
      print("------------------------------------\n")
      tuner.results_summary(num_trials=1)

      f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(n_rounds) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + "Tuning:" + str(tune) +".txt", "a")
      f.write("--------------------------------- \n")
      f.write("TUNER SUMMARY: \n" + str(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values))
      f.write("--------------------------------- \n")
      f.close()


      best_model = tuner.hypermodel.build(best_hps)




  accs = []
  tprs = []
  tnrs = []


  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(n_rounds) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + "Tuning:" + str(tune) +".txt", "a")
  f.write("\n")
  print("TRAINING STARTS")
  f.write("--------------------------------- \n")
  f.write("TRAINING STARTS!\n")
  f.write("--------------------------------- \n")
  f.close()

  for i in range(0, reps):

      distinguisher = None
      if tune is True:
        distinguisher = tuner.hypermodel.build(best_hps)
      else:
        set_global_variables(cipher = cipher, data_format = data_format, network = network, shape = shape, tune = tune, num_epochs = num_epochs)
        distinguisher = create_distinguisher(hp = None)

      x_train, y_train = make_train_data(n_samples = n_train_samples, cipher = cipher, diff = diff, data_format = data_format)
      x_val, y_val     = make_train_data(n_samples = n_val_samples,   cipher = cipher, diff = diff, data_format = data_format)
      x_test, y_test   = make_train_data(n_samples = n_test_samples,  cipher = cipher, diff = diff, data_format = data_format)

      stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience= 3, restore_best_weights= True)
      distinguisher.fit(x_train, y_train, epochs= num_epochs, batch_size=batch_size, validation_data=(x_val, y_val), callbacks=[stop_early])

      (acc, tpr, tnr) = evaluate(distinguisher, x_test, y_test)
      accs.append(acc)
      tprs.append(tpr)
      tnrs.append(tnr)

      f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(n_rounds) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + "Tuning:" + str(tune) +".txt", "a")
      f.write("Trial:" + str(i) + " - " + "ACC:" + str(acc) + "  " + "TPR:" + str(tpr) + "  " + "TNR:" + str(tnr) + "\n")
      f.close()



  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(n_rounds) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + "Tuning:" + str(tune) +".txt", "a")
  f.write("\n")

  print("Accs list:")
  print(accs)
  f.write("Accs list: ")
  f.write(str(accs))
  f.write("\n")
  f.close()



  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(n_rounds) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + "Tuning:" + str(tune) +".txt", "a")
  print("Tprs list:")
  print(tprs)
  f.write("Tprs list: ")
  f.write(str(tprs))
  f.write("\n")
  f.close()


  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(n_rounds) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + "Tuning:" + str(tune) +".txt", "a")
  print("Tnrs list:")
  print(tnrs)
  f.write("Tnrs list: ")
  f.write(str(tnrs))
  f.write("\n")
  f.write("\n")

  f.close()



  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(n_rounds) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + "Tuning:" + str(tune) +".txt", "a")
  f.write("\n")

  f.write("RESULTS: \n" +
            "Acc: " + str(np.mean(accs)) + str(" +- ") + str(np.std(accs)) + str("\n") +
            "Tpr:" + str(np.mean(tprs)) + str(" +- ") + str(np.std(tprs)) + str("\n") +
            "Tnr:" + str(np.mean(tnrs)) + str(" +- ") + str(np.std(tnrs)) + str("\n"))
  f.close()

  print("RESULTS: \n" +
            "Acc: " + str(np.mean(accs)) + str(" +- ") + str(np.std(accs)) + str("\n") +
            "Tpr:" + str(np.mean(tprs)) + str(" +- ") + str(np.std(tprs)) + str("\n") +
            "Tnr:" + str(np.mean(tnrs)) + str(" +- ") + str(np.std(tnrs)) + str("\n"))



  print("RESET GLOBAL VARIABLES")
  reset_global_variables()
  print_global_variables()

# Results

## Questions 1 and 2

Next 3 experiments use Gohrs ND for training on INSHAPE data


### Train on ciphertext differences

In [ ]:
train_and_evaluate(experiment_name = "E1a_Ciphertext_Differences", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Diffs")

In [ ]:
train_and_evaluate(experiment_name = "E1a_Ciphertext_Differences", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Diffs")

In [ ]:
train_and_evaluate(experiment_name = "E1a_Ciphertext_Differences", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Diffs")

In [ ]:
train_and_evaluate(experiment_name = "E1a_Ciphertext_Differences", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Diffs")

In [ ]:
train_and_evaluate(experiment_name = "E1a_Ciphertext_Differences", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Diffs")

In [ ]:
train_and_evaluate(experiment_name = "E1a_Ciphertext_Differences", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Diffs")

### Train on ciphertext pairs

In [ ]:
train_and_evaluate(experiment_name = "E1b_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs")

In [ ]:
train_and_evaluate(experiment_name = "E1b_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs")

In [ ]:
train_and_evaluate(experiment_name = "E1b_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 7, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs")

In [ ]:
train_and_evaluate(experiment_name = "E1b_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs")

In [ ]:
train_and_evaluate(experiment_name = "E1b_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs")

In [ ]:
train_and_evaluate(experiment_name = "E1b_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs")

### Train on ciphertext pairs + info from previous round

In [ ]:
train_and_evaluate(experiment_name = "E2_Ciphertext_PairsPlus", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "PairsPlus")

In [ ]:
train_and_evaluate(experiment_name = "E2_Ciphertext_PairsPlus", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "PairsPlus")

In [ ]:
train_and_evaluate(experiment_name = "E2_Ciphertext_PairsPlus", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "PairsPlus")

In [ ]:
train_and_evaluate(experiment_name = "E2_Ciphertext_PairsPlus", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "PairsPlus")

In [ ]:
train_and_evaluate(experiment_name = "E2_Ciphertext_PairsPlus", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs") #Previous round is just shifted bits so no impact

In [ ]:
train_and_evaluate(experiment_name = "E2_Ciphertext_PairsPlus", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "PairsPlus")

## Question 3 and 4

### Different networks trained on ciphertext pairs with NO tuning

#### Gohr

SHAPED: Networks on ciphertext pairs + no hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = False)

UNSHAPED: Networks on ciphertext pairs + no hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "Gohr", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs",  cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = False)

#### MLP

SHAPED: Networks on ciphertext pairs + no hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = False)

UNSHAPED: Networks on ciphertext pairs + hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = False)

#### Gohr with Inception block in 1st part

SHAPED: Networks on ciphertext pairs + no hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = False)

UNSHAPED: Networks on ciphertext pairs + no hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = False)

#### Gohr with Inception block in 2nd part

SHAPED: Networks on ciphertext pairs + no hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = False)

UNSHAPED: Networks on ciphertext pairs + no hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = False)

### Different networks trained on ciphertext pairs WITH tuning

#### GOHR

SHAPED: Networks on ciphertext pairs + hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "Gohr", shape = "INSHAPE", tune = True)

UNSHAPED: Networks on ciphertext pairs + hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "Gohr", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs",  cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs",  network = "Gohr", shape = "UNSHAPED", tune = True)

#### MLP

SHAPED: Networks on ciphertext pairs + hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "MLP", shape = "INSHAPE", tune = True)

UNSHAPED: Networks on ciphertext pairs + hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "MLP", shape = "UNSHAPED", tune = True)

#### Gohr with Inception block in 1st part

SHAPED: Networks on ciphertext pairs + hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "INSHAPE", tune = True)

UNSHAPED: Networks on ciphertext pairs + hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs",network = "GohrInception1stB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "GohrInception1stB", shape = "UNSHAPED", tune = True)

#### Gohr with Inception block in 2nd part

SHAPED: Networks on ciphertext pairs + hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E4_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "INSHAPE", tune = True)

UNSHAPED: Networks on ciphertext pairs + hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = True)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "GohrInception2ndB", shape = "UNSHAPED", tune = True)

### Special cipher agnostic network DBIT trained on pairs (as in the original paper)

No tuning + No shaping as per original paper

UNSHAPED: Networks on ciphertext pairs + no hyperparameter tuning

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Speck, n_rounds = 6, diff = [0x40, 0], reps = 10, data_format = "Pairs", network = "DBIT", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Simon, n_rounds = 9, diff = [0, 0x400], reps = 10, data_format = "Pairs", network = "DBIT", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Skinny, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], reps = 10, data_format = "Pairs", network = "DBIT", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Katan, n_rounds = 50, diff = [0x4000], reps = 10, data_format = "Pairs", network = "DBIT", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = Present, n_rounds = 6, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], reps = 10, data_format = "Pairs", network = "DBIT", shape = "UNSHAPED", tune = False)

In [ ]:
train_and_evaluate(experiment_name = "E3_Ciphertext_Pairs", cipher_class = ChaCha, n_rounds = 3, diff = [0, 0, 0, 0x8000], reps = 10, data_format = "Pairs", network = "DBIT", shape = "UNSHAPED", tune = False)

## Question 5 and 6

### Updated interfaces of the above neural networks

In [ ]:
import numpy as np
from pickle import dump
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, Lambda, concatenate
from tensorflow.keras.regularizers import l2



# +++ Based on train_nets.py by Gohr +++

bs = 5000
wdir = './freshly_trained_nets/'


def cyclic_lr(num_epochs, high_lr, low_lr):
    return lambda i: low_lr + ((num_epochs - 1) - i % num_epochs) / (num_epochs - 1) * (high_lr - low_lr)

class MyLRSchedule(keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, initial_learning_rate, final_learning_rate, num_epochs):
        self.initial_learning_rate = initial_learning_rate
        self.final_learning_rate = final_learning_rate
        self.num_epochs = num_epochs

    def __call__(self, step):
        return self.final_learning_rate + ((self.num_epochs - 1) - step % self.num_epochs) / (self.num_epochs - 1) * (self.initial_learning_rate - self.final_learning_rate)



def make_resnet(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):

    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)

    train_shape = None
    search_ks = None
    ks1 = 1
    ks = 3

    if shape == "UNSHAPED":
      train_shape = Reshape((num_blocks * word_size * 2, 1))(inp)
      search_ks = num_blocks * word_size
    else:
      train_shape = perm
      search_ks = word_size





    if tune is True:
      reg_param = hp.Float("reg_param", 0.000000001 , 0.0001, step=0.000000001, default = reg_param)
      #ks1 = hp.Int('ks1', min_value=1, max_value= search_ks, step=1, default = ks1)
      ks = hp.Int('ks', min_value=1, max_value= search_ks, step=1, default = ks)
      high_lr = hp.Float("high_lr", 0.0001 , 0.01, step=0.0001, default = high_lr)
      low_lr = hp.Float("low_lr", 0.000001 , 0.001, step=0.000001, default = low_lr)





    conv0 = Conv1D(32, kernel_size= ks1, padding='same', kernel_regularizer=l2(reg_param))(train_shape)
    conv0 = BatchNormalization()(conv0)
    conv0 = Activation('relu')(conv0)

    shortcut = conv0
    for i in range(10):
        conv1 = Conv1D(32, kernel_size= ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation('relu')(conv1)
        conv2 = Conv1D(32, kernel_size= ks, padding='same', kernel_regularizer=l2(reg_param))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)
        shortcut = Add()([shortcut, conv2])

    flat1 = Flatten()(shortcut)

    dense1 = Dense(64, kernel_regularizer=l2(reg_param))(flat1)
    dense1 = BatchNormalization()(dense1)
    dense1 = Activation('relu')(dense1)

    dense2 = Dense(64, kernel_regularizer=l2(reg_param))(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Activation('relu')(dense2)

    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(dense2)
    model = Model(inputs=inp, outputs=out)

    #Add this everywhere and remove from main
    model.compile(optimizer= tf.keras.optimizers.Adam(amsgrad=True), loss='mse', metrics=['acc'])

    return model



def make_MLP(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):

    chunks = 0

    #Add variant for False with INSHAPE
    special_treatment_when_input_is_INSHAPE = True

    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
      chunks = num_blocks
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
      chunks = num_blocks * 2
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)
      chunks = num_blocks * 4


    if tune is True:
      reg_param = hp.Float("reg_param", 0.000000001 , 0.0001, step=0.000000001, default = reg_param)
      high_lr = hp.Float("high_lr", 0.0001 , 0.01, step=0.0001, default = high_lr)
      low_lr = hp.Float("low_lr", 0.000001 , 0.001, step=0.000001, default = low_lr)




    outputs = []
    combined = None





    if shape == "UNSHAPED":

      dense = Dense(chunks * word_size, kernel_regularizer=l2(reg_param))(inp)
      dense = BatchNormalization()(dense)
      dense = Activation('relu')(dense)

      dense = Dense(chunks * word_size, kernel_regularizer=l2(reg_param))(dense)
      dense = BatchNormalization()(dense)
      dense = Activation('relu')(dense)

      combined = dense

    else: #For shape a simple MLP that takes reshaped input does not work - but when given the full unshaped input a simple MLP does
          #To boost the performance we do this special thing to train on reshaped input

      if special_treatment_when_input_is_INSHAPE is True:
          for chunk in list(range(0,chunks)):
              dense = Dense(word_size, kernel_regularizer=l2(reg_param))(perm[:, :, chunk])
              dense = BatchNormalization()(dense)
              dense = Activation('relu')(dense)

              dense = Dense(word_size, kernel_regularizer=l2(reg_param))(dense)
              dense = BatchNormalization()(dense)
              dense = Activation('relu')(dense)
              outputs.append(dense)

          # Concatenate the outputs of the four MLPs
          combined = keras.layers.concatenate(outputs)
      else:
        dense = Dense(word_size, kernel_regularizer=l2(reg_param))(perm)
        dense = BatchNormalization()(dense)
        dense = Activation('relu')(dense)

        dense = Dense(word_size, kernel_regularizer=l2(reg_param))(dense)
        dense = BatchNormalization()(dense)
        dense = Activation('relu')(dense)

        combined = dense



    num_layers_init =  3
    #size = [64, 64, 32]
    size_init = [chunks * word_size, chunks * word_size, int(chunks * word_size/2)]

    if tune is True:
      num_layers = hp.Int('num_layers', min_value=1, max_value= chunks*word_size, step=1, default = num_layers_init)
    else:
        num_layers = num_layers_init

    x = combined
    counter = 0
    for i in range(0, num_layers):
      if tune is True:
        size = hp.Int('size_layer'+str(counter), min_value=1, max_value= chunks*word_size * 4, step=1)
        x = Dense(size, kernel_regularizer=l2(reg_param))(x)
      else:
        size = size_init
        x = Dense(size[counter], kernel_regularizer=l2(reg_param))(x)


      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      counter = counter + 1

    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(x)
    model = Model(inputs=inp, outputs=out)

    model.compile(optimizer= tf.keras.optimizers.Adam(amsgrad=True), loss='mse', metrics=['acc'])


    return model



def make_Gohr_Inception_1stB(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):

    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)

    train_shape = None
    search_ks = None
    num_filters=32

    ks1 = 1
    ks2 = 3
    ks3 = 5
    ks4 = 7
    ks = 3

    if shape == "UNSHAPED":
      train_shape = Reshape((num_blocks * word_size * 2, 1))(inp)
      search_ks = num_blocks * word_size
    else:
      train_shape = perm
      search_ks = word_size




    if tune is True:
      reg_param = hp.Float("reg_param", 0.000000001 , 0.0001, step=0.000000001, default = reg_param)
      #ks1 = hp.Int('ks1', min_value=1, max_value= search_ks, step=1, default = ks1)
      ks2 = hp.Int('ks2', min_value=1, max_value= search_ks, step=1, default = ks2)
      ks3 = hp.Int('ks3', min_value=1, max_value= search_ks, step=1, default = ks3)
      ks4 = hp.Int('ks4', min_value=1, max_value= search_ks, step=1, default = ks4)
      ks = hp.Int('ks', min_value=1, max_value= search_ks, step=1, default = ks)
      high_lr = hp.Float("high_lr", 0.0001 , 0.01, step=0.0001, default = high_lr)
      low_lr = hp.Float("low_lr", 0.000001 , 0.001, step=0.000001, default = low_lr)

    conv01 = Conv1D(num_filters, kernel_size=ks1, padding='same', kernel_regularizer=l2(reg_param))(perm)
    conv02 = Conv1D(num_filters, kernel_size=ks2, padding='same', kernel_regularizer=l2(reg_param))(perm)
    conv03 = Conv1D(num_filters, kernel_size=ks3, padding='same', kernel_regularizer=l2(reg_param))(perm)
    conv04 = Conv1D(num_filters, kernel_size=ks4, padding='same', kernel_regularizer=l2(reg_param))(perm)

    c2 = keras.layers.concatenate([conv01, conv02, conv03, conv04], axis=-1)
    conv0 = BatchNormalization()(c2)
    conv0 = Activation('relu')(conv0)

    shortcut = conv0


    for i in range(10):
        conv1 = Conv1D(num_filters*4, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation('relu')(conv1)
        conv2 = Conv1D(num_filters*4, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)
        shortcut = Add()([shortcut, conv2])
        ks += 2


    flat1 = Flatten()(shortcut)
    dense0 = keras.layers.Dropout(0.8)(flat1) #Dropout added to reduce overfitting - as mentioned in paper
    dense0 = Dense(512, kernel_regularizer=l2(reg_param))(dense0)

    # dense0 = Dense(512, kernel_regularizer=l2(reg_param))(flat1)

    dense0 = BatchNormalization()(dense0)
    dense0 = Activation('relu')(dense0)
    dense1 = Dense(64, kernel_regularizer=l2(reg_param))(dense0)
    dense1 = BatchNormalization()(dense1)
    dense1 = Activation('relu')(dense1)
    dense2 = Dense(64, kernel_regularizer=l2(reg_param))(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Activation('relu')(dense2)
    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(dense2)
    model = Model(inputs=inp, outputs=out)

    model.compile(optimizer= tf.keras.optimizers.Adam(amsgrad=True), loss='mse', metrics=['acc'])


    return(model)




def make_Gohr_Inception_2ndB(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):

    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)

    train_shape = None
    search_ks = None
    num_filters=32
    ks1 = 1
    ks01 = 1
    ks02 = 2
    ks03 = 4
    ks04 = 8
    ks2 = 1


    if shape == "UNSHAPED":
      train_shape = Reshape((num_blocks * word_size * 2, 1))(inp)
      search_ks = num_blocks * word_size
    else:
      train_shape = perm
      search_ks = word_size


    if tune is True:
      reg_param = hp.Float("reg_param", 0.000000001 , 0.0001, step=0.000000001, default = reg_param)
      #ks1 = hp.Int('ks1', min_value=1, max_value= search_ks, step=1, default = ks1)
      ks01 = hp.Int('ks01', min_value=1, max_value= search_ks, step=1, default = ks01)
      ks02 = hp.Int('ks02', min_value=1, max_value= search_ks, step=1, default = ks02)
      ks03 = hp.Int('ks03', min_value=1, max_value= search_ks, step=1, default = ks03)
      ks04 = hp.Int('ks04', min_value=1, max_value= search_ks, step=1, default = ks04)
      #ks2 = hp.Int('ks2', min_value=1, max_value= search_ks, step=1, default = ks2)

      high_lr = hp.Float("high_lr", 0.0001 , 0.01, step=0.0001, default = high_lr)
      low_lr = hp.Float("low_lr", 0.000001 , 0.001, step=0.000001, default = low_lr)

    conv0 = Conv1D(3 * num_filters, kernel_size=ks1, padding='same', kernel_regularizer=l2(reg_param))(train_shape)
    conv0 = BatchNormalization()(conv0)
    conv0 = Activation('relu')(conv0)

    shortcut = conv0
    for i in range(10):
        conv = Conv1D(3 * num_filters, kernel_size=ks01, padding='same', kernel_regularizer=l2(reg_param))(shortcut)

        conv01 = Conv1D(num_filters, kernel_size=ks02, padding='same', kernel_regularizer=l2(reg_param))(conv)
        conv02 = Conv1D(num_filters, kernel_size=ks03, padding='same', kernel_regularizer=l2(reg_param))(conv)
        conv03 = Conv1D(num_filters, kernel_size=ks04, padding='same', kernel_regularizer=l2(reg_param))(conv)

        concat = keras.layers.concatenate([conv01, conv02, conv03], axis=-1)
        concat = BatchNormalization()(concat)
        concat = Activation('relu')(concat)

        conv2 = Conv1D(3 * num_filters, kernel_size=ks2, padding='same', kernel_regularizer=l2(reg_param))(concat)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)
        shortcut = Add()([shortcut, conv2])

    flat = Flatten()(shortcut)

    #pool = keras.layers.GlobalAveragePooling1D()(flat)

    dense1 = Dense(64, kernel_regularizer=l2(reg_param))(flat)
    dense1 = BatchNormalization()(dense1)
    dense1 = Activation('relu')(dense1)

    dense2 = Dense(64, kernel_regularizer=l2(reg_param))(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Activation('relu')(dense2)

    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(dense2)
    model = Model(inputs=inp, outputs=out)

    model.compile(optimizer= tf.keras.optimizers.Adam(amsgrad=True), loss='mse', metrics=['acc'])


    return model




def get_dilation_rates(input_size):
    """Helper function to determine the dilation rates of DBitNet given an input_size. """
    drs = []
    while input_size >= 8:
        drs.append(int(input_size / 2 - 1))
        input_size = input_size // 2

    return drs


def make_DBIT(hp, tune, data_format, shape, num_blocks, word_size, high_lr, low_lr, reg_param, num_epochs):
    input_size = None

    #From here, only the input size is taken as the model does not need reshaping
    if data_format == "Diffs":
      inp = Input(shape=(num_blocks * word_size * 1, ))
      rs = Reshape((1 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
      input_size= num_blocks * word_size * 1
    if data_format == "Pairs":
      inp = Input(shape=(num_blocks * word_size * 2, ))
      rs = Reshape((2 * num_blocks, word_size))(inp)
      perm = Permute((2, 1))(rs)
      input_size= num_blocks * word_size * 2
    if data_format == "PairsPlus":
      inp = Input(shape=(num_blocks * word_size * 2  * 2,))
      rs = Reshape((4 * num_blocks  ,  word_size))(inp)
      perm = Permute((2, 1))(rs)
      input_size= num_blocks * word_size * 4


    '''
    train_shape = None
    if shape == "UNSHAPED":
      train_shape = Reshape((num_blocks * word_size * 2, 1))(inp)
    else:
      train_shape = perm
    '''

    n_filters=32
    n_add_filters=16


    # determine the dilation rates from the given input size
    dilation_rates = get_dilation_rates(input_size)

    # prediction head parameters (similar to Gohr)
    d1 = 256 # TODO this can likely be reduced to 64.
    d2 = 64
    reg_param = 1e-5

    # define the input shape
    inputs = Input(shape=(input_size, 1))
    x = inputs

    # normalize the input data to a range of [-1, 1]:
    x = tf.subtract(x, 0.5)
    x = tf.divide(x, 0.5)

    for dilation_rate in dilation_rates:
        ### wide-narrow blocks
        x = Conv1D(filters=n_filters,kernel_size=2,padding='valid', dilation_rate=dilation_rate,strides=1,activation='relu')(x)
        x = BatchNormalization()(x)
        x_skip = x
        x = Conv1D(filters=n_filters,kernel_size=2,padding='causal',dilation_rate=1,activation='relu')(x)
        x = Add()([x, x_skip])
        x = BatchNormalization()(x)

        n_filters += n_add_filters

    ### prediction head
    out = tf.keras.layers.Flatten()(x)

    dense0 = Dense(d1, kernel_regularizer=l2(reg_param))(out);
    dense0 = BatchNormalization()(dense0);
    dense0 = Activation('relu')(dense0);
    dense1 = Dense(d1, kernel_regularizer=l2(reg_param))(dense0);
    dense1 = BatchNormalization()(dense1);
    dense1 = Activation('relu')(dense1);
    dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1);
    dense2 = BatchNormalization()(dense2);
    dense2 = Activation('relu')(dense2);
    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(dense2)

    model = Model(inputs, out)

    model.compile(optimizer= tf.keras.optimizers.Adam(amsgrad=True), loss='mse', metrics=['acc'])


    return model





def create_distinguisher(cipher, data_format, network, shape, tune, num_epochs, hp = None):

    high_lr=0.002
    low_lr=0.0001
    reg_param=0.0001


    # create the network
    net = None

    if network == "Gohr":
        net = make_resnet(hp = hp, tune = False, data_format = data_format, shape = shape, num_blocks = cipher.get_n_words(), word_size = cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = num_epochs)

    if network == "MLP": #TD
        net = make_MLP(hp = hp, tune = False, data_format = data_format, shape = shape, num_blocks = cipher.get_n_words(), word_size = cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = num_epochs)

    if network == "GohrInception1stB":

        net = make_Gohr_Inception_1stB(hp = hp, tune = False, data_format = data_format, shape = shape, num_blocks = cipher.get_n_words(), word_size = cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = num_epochs)

    if network == "GohrInception2ndB":
        net = make_Gohr_Inception_2ndB(hp = hp, tune = False, data_format = data_format, shape = shape, num_blocks = cipher.get_n_words(), word_size = cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = num_epochs)

    if network == "DBIT":
        net = make_DBIT(hp = hp, tune = False, data_format = data_format, shape = shape, num_blocks = cipher.get_n_words(), word_size = cipher.get_word_size(), high_lr = high_lr, low_lr = low_lr, reg_param = reg_param, num_epochs = num_epochs)


    return net

### Strategies

In [ ]:
def freeze_feature_extractor(model):
  for layer in model.layers:
    if layer.name != "Train":
      layer.trainable = False

  return model

In [ ]:
import logging
import pandas as pd
import numpy as np

batch_size = 5000

def train_one_round(model, cipher, diff, data_format):

      x_train, y_train = make_train_data(n_samples = 10**7, cipher = cipher, diff = diff, data_format = data_format)
      x_val, y_val = make_train_data(n_samples = 10**6, cipher = cipher, diff = diff, data_format = data_format)


      stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience= 3, restore_best_weights= True)
      model.fit(x_train, y_train, epochs= 10, batch_size=batch_size, validation_data=(x_val, y_val), callbacks=[stop_early])

      return model

def run_experiment(start_round, end_round, cipher_class, diff, data_format, shape, network, strategy):

    print("Startegy: " + strategy)
    cipher = cipher_class(start_round)
    model = create_distinguisher(cipher_class(start_round), data_format, network, shape = shape, tune = False, num_epochs = None, hp = None) #num_epochs not needed anymore since no cyclical LR

    if strategy == "Iterative":
      for current_round in range(start_round, end_round+1):
        cipher = cipher_class(current_round)
        model = train_one_round(model, cipher, diff, data_format)

    if strategy == "Freeze":
        model = train_one_round(model, cipher, diff, data_format)
        model = freeze_feature_extractor(model)
        model = train_one_round(model, cipher, diff, data_format)


    return model


def repeat_experiment_with_strategy(experiment_name, reps, start_round, end_round, cipher_class , diff, data_format, shape, network, strategy):
  cipher = cipher_class(end_round)
  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(end_round) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format +".txt", "a")
  f.write("Cipher: " + str(cipher_class.get_name()) + " - " + str(end_round) +" Rounds" + " - Blocks: " + str(cipher.get_n_words()) + " - Word size: "+ str(cipher.get_word_size()) + "\n")
  f.write("Input difference: " + str(diff) + "\n")
  f.write("Network: " + network + "\n")
  f.write("Trains on: " + data_format + " - " + shape + "\n")
  f.write("Strategy: " + strategy + "\n")
  f.close()

  accs = []
  tprs = []
  tnrs = []

  for i in range(0, reps):
      model = run_experiment(start_round, end_round, cipher_class, diff, data_format, shape, network, strategy)

      x_test, y_test   = make_train_data(n_samples = 10**6,  cipher = cipher, diff = diff, data_format = data_format)
      (acc, tpr, tnr) = evaluate(model, x_test, y_test)
      accs.append(acc)
      tprs.append(tpr)
      tnrs.append(tnr)

      f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(end_round) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + ".txt", "a")
      f.write("Trial:" + str(i) + " - " + "ACC:" + str(acc) + "  " + "TPR:" + str(tpr) + "  " + "TNR:" + str(tnr) + "\n")
      f.close()



  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(end_round) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format +".txt", "a")
  f.write("\n")

  print("Accs list:")
  print(accs)
  f.write("Accs list: ")
  f.write(str(accs))
  f.write("\n")
  f.close()



  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(end_round) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + ".txt", "a")
  print("Tprs list:")
  print(tprs)
  f.write("Tprs list: ")
  f.write(str(tprs))
  f.write("\n")
  f.close()


  f = open(experiment_name + "_" + cipher_class.get_name() + "_" + str(end_round) + "R" + "_Net:" + network + "_" + shape + "_"+ data_format + ".txt", "a")
  print("Tnrs list:")
  print(tnrs)
  f.write("Tnrs list: ")
  f.write(str(tnrs))
  f.write("\n")
  f.write("\n")

  f.close()



### Question 5: Iterative training strategy

#### Gohr

Input is INSHAPE

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Iterative")

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10 start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Iterative")

#### MLP

Input is INSHAPE

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Iterative")

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Iterative")

#### Gohr with Inception block in 1st part

Input is INSHAPE

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Iterative")

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Iterative")

#### Gohr with Inception block in 2nd part

Input is INSHAPE

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Iterative")

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Iterative")

#### DBit

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Iterative")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E5_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Iterative")

### Question 6: Freezing layer strategy

#### Gohr

Input is INSHAPE

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "INSHAPE", network = "Gohr", strategy = "Freeze")

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "Gohr", strategy = "Freeze")

#### MLP

Input is INSHAPE

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "INSHAPE", network = "MLP", strategy = "Freeze")

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "MLP", strategy = "Freeze")

#### Gohr with Inception block in 1st part

Input is INSHAPE

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception1stB", strategy = "Freeze")

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception1stB", strategy = "Freeze")

#### Gohr with Inception block in 2nd part

Input is INSHAPE

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "INSHAPE", network = "GohrInception2ndB", strategy = "Freeze")

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "GohrInception2ndB", strategy = "Freeze")

#### DBit

Input is UNSHAPED

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Speck , diff = [0x40, 0], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 8, end_round = 9, cipher_class = Simon , diff = [0, 0x400], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Skinny , diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 49, end_round = 50, cipher_class = Katan , diff = [0x4000], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 5, end_round = 6, cipher_class = Present, diff = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0xd, 0, 0, 0, 0, 0, 0], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Freeze")

In [ ]:
repeat_experiment_with_strategy(experiment_name = "E6_Ciphertext_Pairs", reps = 10, start_round = 2, end_round = 3, cipher_class = ChaCha, diff = [0, 0, 0, 0x8000], data_format = "Pairs", shape = "UNSHAPED", network = "DBIT", strategy = "Freeze")

## Question 7

Based on code from: https://github.com/Crypto-TII/nnbits/tree/main

In [ ]:
# USE YOUR HARDWARE INFORMATION TO ADJUST YOUR HARDWARE INFORMATION DOWN BELOW
!nvidia-smi

In [ ]:
!git clone https://github.com/Crypto-TII/nnbits
!pip install -r nnbits/requirements.txt

In [ ]:
# CHOOSE CIPHER, ROUND NR, and INPUT DIFFERENCE

nr = 9
n  = 10**7

cipher = Simon(nr)
diff = [0, 0x400]
data_format = "Pairs"

X, Y = make_train_data(n_samples = n, cipher = cipher, diff = diff, calc_back = 0, data_format = data_format)
X_val, Y_val = make_train_data(n_samples = n//10, cipher = cipher, diff = diff, calc_back = 0, data_format = data_format)

In [ ]:
np.save('data_train_full.npy', X);
np.save('data_train_labels.npy', Y);
np.save('data_test_full.npy', X_val);
np.save('data_test_labels.npy', Y_val);

In [ ]:
import numpy as np

X = np.load('data_train_full.npy')
Y = np.load('data_train_labels.npy')
X_val = np.load('data_test_full.npy')
Y_val = np.load('data_test_labels.npy')

In [ ]:
final   = np.r_[X, X_val]
np.save('combined.npy', final)

In [ ]:
X = X[Y==1]
X_val = X_val[Y_val==1]

final   = np.r_[X, X_val]

np.save('combined_Y1.npy', final)

In [ ]:
# ADJUST NAME

savepath = f'gohr_ensemble_longR8_Simon'

In [ ]:
import toml
import numpy as np
import matplotlib.pyplot as plt

from nnbits.nnbits.filemanager import FileManager
from nnbits.nnbits.bitanalysis import get_X


In [ ]:
F = FileManager(savepath)

datapath = 'combined_Y1.npy'
cfgdict = {'DATAPATH': datapath,
            # ensemble settings
            'NEURAL_NETWORK_MODEL': 'gohr',
            'NEURAL_NETWORKS': 64,   #ADJUST NUMER TO MATCH TO INPUT BITS - for the experiments in the paper, this doesn't need to be changed
            'PREDICT_LABEL': False,
            'SELECT_BITS_STRATEGY': 'target',
            'TARGET_BITS': [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63],  #ADJUST NUMER TO MATCH TO INPUT BITS  - for the experiments in the paper, this doesn't need to be changed
            'INPUT_DATA_OP': 'zero',
            # hardware settings
            # ADJUST HERE ACCORDING TO YOUR HARDWARE
            'N_GPUS': 6,
            'N_ACTORS_PER_GPU': 4,
            'GPU_PER_ACTOR': 0.25,
            'CPU_PER_ACTOR': 5,
            # training settings
            'N_EPOCHS': 200,
            'N_TRAIN': 4997120,
            'N_VAL': 499712,
            'BATCHSIZE': 4096,
            'EARLY_STOPPING': False,
            'SAVE_BEST_WEIGHTS': True}

with open(F.filename_config(), 'w') as configfile:
    toml.dump(cfgdict, configfile)

print("="*len(datapath))
print(datapath)

In [ ]:
# ADJUST NAME

!python -m nnbits.nnbits.run --savepath 'gohr_ensemble_longR8_Simon'

In [ ]:
import matplotlib.pyplot as plt

# Data from above needs to be introduced below in form {bit_number : acc}

test_accuracies_dict = { bit_no: acc} # Complete this list
# Plotting
plt.bar(test_accuracies_dict.keys(), test_accuracies_dict.values())
plt.xlabel('Bit ID')
plt.ylabel('Test accuracy')
plt.title('Test accuracies of ResNet on Simon-8R') #Adjust name
plt.ylim(0, 100)
plt.savefig('Simon-8R.png', dpi=300)  #Adjust name
plt.show()

In [ ]:
F = FileManager(savepath)

datapath = 'combined.npy'
cfgdict = {'DATAPATH': datapath,
            # ensemble settings
            'NEURAL_NETWORK_MODEL': 'gohr',
            'NEURAL_NETWORKS': 64,  #ADJUST NUMER TO MATCH TO INPUT BITS  - for the experiments in the paper, this doesn't need to be changed
            'PREDICT_LABEL': False,
            'SELECT_BITS_STRATEGY': 'target',
            'TARGET_BITS': [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63],  #ADJUST NUMER TO MATCH TO INPUT BITS  - for the experiments in the paper, this doesn't need to be changed
            'INPUT_DATA_OP': 'zero',
            # hardware settings
            # ADJUST HERE ACCORDING TO YOUR HARDWARE
            'N_GPUS': 6,
            'N_ACTORS_PER_GPU': 2,
            'GPU_PER_ACTOR': 0.5,
            'CPU_PER_ACTOR': 5,
            # testing settings
            'N_TEST': 11_000_000,
            'TEST_ONLY': True,
            # training settings
            'N_EPOCHS': 2,
            'N_TRAIN': 0,
            'N_VAL': 0,
            'BATCHSIZE': 4096,
            'EARLY_STOPPING': False,
            'SAVE_BEST_WEIGHTS': True
             }

with open(F.filename_config(), 'w') as configfile:
    toml.dump(cfgdict, configfile)

print("="*len(datapath))
print(datapath)

In [ ]:
# ADJUST NAME

!python -m nnbits.nnbits.run --savepath 'gohr_ensemble_longR8_Simon'

In [ ]:
from nnbits.nnbits.gohr_original.gohr import bs, LearningRateScheduler, cyclic_lr

def create_model(d1=64, d2=64, reg_param=1e-5):

    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices("GPU")
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    from keras.models import Model
    from keras.layers import Dense, Conv1D, Conv2D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, \
        Activation, Lambda
    from keras.layers import Concatenate, MaxPooling2D
    from keras.regularizers import l2
    from keras import backend as K
    import numpy as np

    inp = Input(shape=(64,))
    dense1 = Dense(d1,kernel_regularizer=l2(reg_param))(inp);
    dense1 = BatchNormalization()(dense1);
    dense1 = Activation('relu')(dense1);
    dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1);
    dense2 = BatchNormalization()(dense2);
    dense2 = Activation('relu')(dense2);
    out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param))(dense2);
    model = Model(inputs=inp, outputs=out);
    return(model);

N_TRAIN = 10_000_000
N_EVAL = 1_000_000

In [ ]:


Y = np.load('data_train_labels.npy')[:N_TRAIN]

X = np.zeros((N_TRAIN, 64))

for network_id in np.arange(64):
    filename = f'{savepath}/test_accuracies_bit_by_bit/{network_id}_testing.npy'
    x = np.load(filename)
    X[:,network_id] = x[:N_TRAIN]


Y_eval = np.load('data_test_labels.npy')[:N_EVAL]

X_eval = np.zeros((N_EVAL, 64))

for network_id in np.arange(64):

    filename = f'{savepath}/test_accuracies_bit_by_bit/{network_id}_testing.npy'
    x = np.load(filename)
    X_eval[:,network_id] = x[N_TRAIN:]

num_epochs = 200

model = create_model(reg_param=10**-5);
model.compile(optimizer='adam',loss='mse',metrics=['acc']);

lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));

h = model.fit(X, Y, epochs=num_epochs, batch_size=bs, callbacks=[lr], validation_data=(X_eval, Y_eval))

print("Best validation accuracy: ", np.max(h.history['val_acc']));